## Machine Learning with Python


### Template to build a machine learning model with Python
1. This template contains detailed steps to preprocess raw data and convert it into model input
2. This template uses Random Forest, Gradient Boosting Machine, and lightGBM algorithms

### Steps to build a model:
1. Load the libaries and data
2. Data preprocessing  
    i. Remove variables with high missing values percentage  
    ii. Remove categorical variables with many levels  
    iii. Convert categorical variables to binary variables  
    iv. Split train and out of time samples  
    v. Impute missing values  
    vi. Remove variables based on Gini threshold  
3. Train the model (currently supporting scikit-learn RF, scikit-learn GBM, H2O RF, lightGBM)  
    i. Background feature selection  
    ii. Grid search for hyperparameter optimization using the following methods: \
        a. iterative steps \
        b. search through fixed grid space \
        c. search through random grid space \
        d. search using Optuna \
    iii. Train machine learning model   
    iv. Calculate feature importance  
4. Evaluate model  
    i. Produce evaluation report  
    ii. Produce lifting table \
    iii. Produce evaluation graphs

### Machine Learning Libaries:
1. preprocessing
2. feature_selection
3. model_builder
4. evaluation


In [ ]:
# Make the notebook full screen
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import os
import pandas as pd
import importlib
import sys 

if sys.version_info[:3] < (3,4):
    os.getcdw()
    code_dir = os.path.dirname(os.getcdw())
    project_dir = os.path.dirname(os.path.dirname(os.getcdw()))
    data_path = os.path.join(code_dir, "data")
    functions_path = os.path.join(project_dir, "functions")
else: 
    from pathlib import Path
    current_directory = os.path.dirname(Path.cwd())
    code_dir = os.path.dirname(os.path.dirname(current_directory))
    project_dir = os.path.join(code_dir, "2_Supervised_Modeling\\Machine_Learning")
    data_path = os.path.join(code_dir, "2_Supervised_Modeling\\Machine_Learning\\data")
    functions_path = os.path.join(code_dir, 'functions')

print(code_dir)
print(project_dir)
print(data_path)
print(functions_path)

In [ ]:
# General Python modules
import time
import json

In [ ]:
# Set the path for the library
import sys
sys.path.insert(0, functions_path)
import data_transformation as dtran
import variable_reduction as vr
import feature_elimination as fe
import model_builder as mb
import reports as rp
import useful_functions as ufun
from load_data import load_data
import keras_functions as ks_fn

In [ ]:
pd.set_option('display.max_columns', 100)

# Initialize the solution variables

In [ ]:
with open(os.path.join(project_dir, 'data/input/Supervised_Modeling_Solution_Input_ML.json')) as f:
    inputs = json.load(f)

In [ ]:
inputs

## Essential parameters

In [ ]:
# String. Specify how to load the data. Options: csv, parq.
Load_from = inputs["Load_from"]
# String. Specify the data location: this is the folder where the data for this project are saved. 
data_location = inputs["data_location"]
# String. Set the input data file. 
table_name = inputs["table_name"]
# Float. Number between 0-1 determining what percent of data to subsample. 
sample = float(inputs["sample"])
# String. Set the target variable name in the original dataset. 
target_variable_name = inputs["target_variable_name"]
# String. Set the weight variable name in the original dataset. If not avaulable, then provide "None".
weight_variable_name = inputs["weight_variable_name"]
# String. Set the sample column that has sample information, e.g. train/test/OOT or segment information, and will be used to split the data in different samples
# String. If this column does not exist, then provide "None".
sample_variable_name = inputs["sample_variable_name"]
# String. Name of the amount/cost per record. If this column does not exist, then provide "None".
amount_variable_name = inputs["amount_variable_name"]
# List of strings. Set the sub-sample values that are in the sample_variable_name field, e.f. for train/test data split and/or for different segments. 
# All samples defined in this parameters will be picked up by the solution and results will be created for these samples. 
# If sample column does not exist, then provide '[None]' (without quotes).
sample_values = inputs["sample_values"]
# List. Provide the feature names for the numeric variables that will be used for modeling. 
original_candidate_variables_numeric = inputs["numeric_variables_modeling"]
# List. Provide the feature names for the character variables that will be used for modeling. 
original_candidate_variables_character = inputs["character_variables_modeling"]

## Advanced parameters

In [ ]:
# Float. Takes values between 0 and 1. Used in 'select_missing_variables_to_drop' function. Variables with percentage missing values above this threshold will be 
# dropped from the rest of the process. 
select_missing_variables_to_drop_threshold = inputs["select_missing_variables_to_drop_threshold"]
# Integer. Used in 'character_classification' function. Character variables with more levels than this threshold will be dropped from the rest of the process. 
character_classification_threshold = inputs["character_classification_threshold"]
# Float. Used in the 'replace_outliers' function in the outlier removal section. This is the coefficient for Interquantile range. 
# It can be used to adjust how many outliers to replace; the higher the value the less outliers are replaced. 
iqr_coef = inputs["iqr_coef"]
# String. Used in 'impute_missing' class. Select the stratefy to impute the missing values. Current options are "median", "mean", 
# or a specific value without quotes, e.g. 0.
impute_missing_imputation_strategy = inputs["impute_missing_imputation_strategy"]
# Float. Variables with Gini coefficient below this threshold will be dropped from the reamained of the analysis. 
gini_threshold = inputs["gini_threshold"]
# Integer. Number of top features based on feature importance to apply a first screening using Random Forest. 
rf_initial_screening_var_threshold = inputs["rf_initial_screening_var_threshold"]
# Integer. Number of features to remain using backward selection with random forest model: In each step, a feature that has the minimum Gini is selected to be removed. 
rf_backward_selection_var_threshold = inputs["rf_backward_selection_var_threshold"]

# Load the data

In [ ]:
data_full = load_data(method = Load_from, 
                     data_path = data_location, 
                     table_name = table_name, 
                     sample = sample)

In [ ]:
data_full.info()

In [ ]:
data_full.head()

# Create the Weight, Sample and Amount variables, if not available in the input dataset

In [ ]:
# Create the weight variable, if it doesn't exist.
data_full, weight_variable_name_solution = dtran.weight_var_assignment(input_data = data_full, 
                                                                                     weight_variable = weight_variable_name)

# Create the sample variable, if it doesn't exist.
data_full, sample_values_solution, sample_variable_name_solution = dtran.sample_var_assignment(input_data = data_full, 
                                                                                        sample_variable = sample_variable_name,
                                                                                        sample_values = sample_values)

# Create the amount variable, if it doesn't exist.
data_full, amount_variable_name_solution = dtran.amount_var_assignment(input_data = data_full, 
                                                                                     amount_variable = amount_variable_name)

# Subset the dataset to use only the samples selected by 'sample values'

In [ ]:
data_full = data_full[data_full[sample_variable_name_solution].isin(sample_values_solution)]

# Convert variable data types based on user information

In [ ]:
# Convert character variables
data_full, character_variables_list = dtran.convert_character_var(input_data = data_full, 
                                                        character_variables = original_candidate_variables_character,
                                                        sample_variable = sample_variable_name_solution)

# Convert numeric variables
data_full, numeric_variables_list = dtran.convert_numeric_var(input_data = data_full, 
                                                        numeric_variables = original_candidate_variables_numeric,
                                                        weight_variable = weight_variable_name_solution, 
                                                        amount_variable = amount_variable_name_solution, 
                                                        target_variable = target_variable_name)

# Data quality report

In [ ]:
# Create folder, if it doesn't exist
ufun.create_folder(data_path = data_path, 
                   folder_name = 'output')

In [ ]:
dq = rp.dq_report(input_data = data_full, 
                data_path = data_path, 
                variables = character_variables_list + numeric_variables_list, 
                weight_variable = weight_variable_name_solution, 
                dq_report_file = 'data_quality_report.csv')

# Split sample data

In [ ]:
data, sample_values_dict = dtran.split_sample_data(
    input_data=data_full, 
    sample_values_solution=sample_values_solution, 
    sample_variable_name_solution=sample_variable_name_solution
    )

# Set the original candidate variables

In [ ]:
original_candidate_variables = original_candidate_variables_character + original_candidate_variables_numeric
print(ufun.color.BLUE + 'Original candidate variables: ' + ufun.color.END + str(original_candidate_variables))

# Remove variables with high missing values percentage

In [ ]:
# Variables excluded from the non-predictive features: keys, target, sample, etc
excluded_variables = [x for x in data['data_{}'.format(sample_values_solution[0])].columns if x not in original_candidate_variables]
print(ufun.color.BLUE + 'Variables to be excluded: ' + ufun.color.END + str(excluded_variables))
print()
# Produce and save the missing values table to review
missing_variables_table, missing_variables = vr.missing_values_vars(
    sample_values_dict=sample_values_dict, 
    data_path=data_path, 
    input_data=data, 
    weight_variable_name_solution=weight_variable_name_solution, 
    select_missing_variables_to_drop_threshold=select_missing_variables_to_drop_threshold
    )
# Create the variables to remove: non-predictors + variables with too many missing information
excluded_variables = excluded_variables + missing_variables
print(ufun.color.BLUE + 'Variables to remove from the remainder of the analysis: ' + ufun.color.END + str(excluded_variables))

# Remove character variables with many levels

In [ ]:
keep_char_vars_levels, excl_char_vars = vr.character_var_levels(
    input_data = data, 
    data_path = data_path, 
    sample_values_solution = sample_values_solution,
    excluded_variables = excluded_variables, 
    character_classification_threshold = character_classification_threshold
    )

# Outlier replacement for numeric variables

In [ ]:
outlier_variables = [i for i in original_candidate_variables_numeric if i not in excluded_variables]
data_full = dtran.replace_outliers(
    input_data = data_full, 
    variables = outlier_variables, 
    weight_variable = weight_variable_name_solution, 
    data_path = data_path, 
    outlier_info_file = 'outlier_info.csv', 
    iqr_coef = iqr_coef
    )

In [ ]:
# Split sample data
data, temp_dict = dtran.split_sample_data(
    input_data=data_full, 
    sample_values_solution=sample_values_solution, 
    sample_variable_name_solution=sample_variable_name_solution
    )

# Convert categorical variables to binary variables

In [ ]:
data_full = dtran.character_to_binary(
    input_data = data_full, 
    input_variable_list = keep_char_vars_levels, 
    drop = 'last', # Specifies which value to drop from the one hot encoder. None will return binary variables for all categories. 'first' will drop the most populated category. 'last' will drop the less populated category. 
    protected_class_valid_values = None # Specifies accepted values for the protected class column. For non-protected class conversions use 'None'
    )

In [ ]:
# Split sample data
data, temp_dict = dtran.split_sample_data(
    input_data=data_full, 
    sample_values_solution=sample_values_solution, 
    sample_variable_name_solution=sample_variable_name_solution
    )

In [ ]:
# Keep all numeric variables, including those that were one-hot encoded
keep_num_vars = ufun.identify_numeric_variables(input_data=data['data_{}'.format(sample_values_solution[0])])
keep_num_vars = [x for x in keep_num_vars if x not in excluded_variables]
print('Keeping the following variables: ', keep_num_vars)
print(len(keep_num_vars))

# Impute missing values

In [ ]:
variables_with_missing_dict = vr.select_missing_variables_to_drop_dict(
    sample_values_dict = sample_values_dict, 
    data_path = data_path)

In [ ]:
# Select numeric features with missing values. Imputation will be applied to only these features, in order to improve the performance of the code. 
variables_with_missing = list(dict.fromkeys(sum(variables_with_missing_dict.values(), [])))
num_variables_with_missing = [i for i in keep_num_vars if i in variables_with_missing]
num_variables_with_missing

In [ ]:
# Impute missing values
start_time = time.time()
impute_missing = dtran.impute_missing(
        variables = num_variables_with_missing, 
        imputation_strategy = impute_missing_imputation_strategy)
impute_missing.imputation_fit_weight(
        input_data = data['data_{}'.format(sample_values_solution[0])], 
        weight_variable = weight_variable_name_solution)

for i, j in sample_values_dict.items():
    impute_missing.imputation_transform(input_data = data['data_{}'.format(i)])

print('This code took %.2fs. to run'%(time.time() - start_time))

In [ ]:
# Check missing values for imputed variables
for i, j in sample_values_dict.items():
    start_time = time.time()
    print(ufun.color.BOLD + ufun.color.PURPLE + ufun.color.UNDERLINE + 'SAMPLE ' + i + ufun.color.END)

    if num_variables_with_missing != []:
        print(data['data_{}'.format(i)][num_variables_with_missing].apply
              (lambda x: (sum(data['data_{}'.format(i)][x.isnull()][weight_variable_name_solution])
                /sum(data['data_{}'.format(i)][weight_variable_name_solution])) * 100, axis=0).sort_values(ascending=False))
    else: 
        print('There are no variables with missing values to impute')

    print('This code took %.2fs. to run'%(time.time() - start_time))

# Drop numeric variables with only one value

In [ ]:
keep_num_vars_one_v = vr.keep_num_variables_one_value(
    keep_num_vars = keep_num_vars, 
    data_path = data_path, 
    dq_report = 'data_quality_report.csv'
    )

# Drop variables based on low Gini

In [ ]:
gini_table = fe.gini_values_weight(feats = keep_num_vars_one_v, 
                   input_data = data['data_{}'.format(sample_values_solution[0])], 
                   target_variable = target_variable_name, 
                   weight_variable = weight_variable_name_solution, 
                   data_path = data_path, 
                   gini_info_file = 'gini_info.csv', 
                   n_bands = 10)
keep_num_vars_gini = list(gini_table.loc[gini_table['Gini coefficient'] >= gini_threshold, 'variable'].values)
print(ufun.color.PURPLE + 'Keeping the following variables with Gini > ' + str(gini_threshold) + ': ' + ufun.color.END + str(keep_num_vars_gini))
print(len(keep_num_vars_gini))

# Random Forest

## Feature selection RF

In [ ]:
# Provide input data to feature selection class
select = fe.SelectBest_weight(df=data['data_{}'.format(sample_values_solution[0])], 
                              target=target_variable_name, 
                              weight=weight_variable_name)

In [ ]:
# Select top features to apply a first screening
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=1234, n_jobs=6)
rf_importance, feats_best_rf = select.top_rf_feat(feats=keep_num_vars_gini, model=model_rf, n=rf_initial_screening_var_threshold)
print('The top Random Forest features are: \n', feats_best_rf)
rf_importance

In [ ]:
# Backward selection with random forest model: In each step a feature with the minimum Gini is selected to be removed.
feats_best_rf_back = select.backward_recur(feats=feats_best_rf, 
                                           oos=data['data_{}'.format(sample_values_solution[1])], 
                                           model=model_rf, 
                                           min_feats=rf_backward_selection_var_threshold, 
                                           classification=True)
print(feats_best_rf_back)

## Grid search and train RF machine learning model

### Grid search based on iterative step

In [ ]:
# Hyperparameter tuning: The process works as follows:
# 1) Iterate n_estimators (grid) for the fixed values of max_depth and max_features (params). Select the value of n_estimators that minimizes the loss function. 
# 2) Keep the n_estimators value from the previous step. Iterate max_depth (grid) for the fixed value of max_features (params). Select the value of max_depth that minimizes the loss function. 
# 3) Keep the max_depth value from the previous step. Iterate max_features (grid). Select the value of max_features that minimizes the loss function. 

import math
# 18 models in total
grid = {'n_estimators':[10, 50, 80, 100, 200, 400],  # Number of trees, 100-500 is usually sufficient /
       'max_depth':[2, 5, 10, 20],  # Depth of the tree, 5-6 is usually default
       'max_features':[*range(1,len(feats_best_rf_back),math.ceil(len(feats_best_rf_back)/5))],  # Number of variables randomly sampled as candidates at each split, sqrt(p) is usually the default
       'min_samples_leaf': [1, 10, 100]} # Minimum number of observations in each leaf, 10 is usually default

params = {'n_estimators': 100, 'max_depth':10, 'max_features':math.ceil(math.sqrt(len(feats_best_rf_back))), 'min_samples_leaf':10}

opt_params_iterative, loss = mb.step_search_weight(estimator=RandomForestClassifier, 
                                                   params=params, 
                                                   grid=grid, 
                                                   target=target_variable_name, 
                                                   weight=weight_variable_name, 
                                                   dev=data['data_{}'.format(sample_values_solution[0])], 
                                                   val=data['data_{}'.format(sample_values_solution[1])], 
                                                   keep=feats_best_rf_back)
print('\n Best Parameters')
print(opt_params_iterative)
print('Loss: ', loss)

# Load RF machine leanring library
from sklearn.ensemble import RandomForestClassifier
import joblib

# Define the random forest model
rf = RandomForestClassifier(n_estimators=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='n_estimators'][0], 
                            max_depth=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='max_depth'][0], 
                            max_features=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='max_features'][0], 
                            min_samples_leaf=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='min_samples_leaf'][0], 
                            random_state=0)

# Train the model
rf_model = mb.fit_model_weight(df=data['data_{}'.format(sample_values_solution[0])], 
                               feats=feats_best_rf_back, 
                               target=target_variable_name, 
                               weight=weight_variable_name, 
                               model=rf, 
                               model_name=data_path + '/output/' + 'rf_iterative.pkl')

# Compute the score for the OOT data
for k in data.keys():
    data[k].loc[:, 'rf_score_iterative_numeric'] = rf_model.predict_proba(data[k].loc[:, feats_best_rf_back].values)[:, 1]
    data[k].loc[:, 'rf_score_iterative_binary'] = list(map(round, data[k].loc[:, 'rf_score_iterative_numeric']))    

### Grid search based on RandomizedSearchCV - also uses cross-validation

In [ ]:
# 1,296 models in total
grid = {'n_estimators':[10, 50, 80, 100, 200, 400],  # Number of trees, 100-500 is usually sufficient /
       'max_depth':[2, 3, 4, 5, 6, 7],  # Depth of the tree, 5-6 is usually default
       'max_features':[*range(15,21,1)],  # Number of variables randomly sampled as candidates at each split, sqrt(p) is usually the default
       'min_samples_leaf': [1, 10, 100], # Minimum number of observations in each leaf, 10 is usually default
       'criterion': ['gini', 'entropy']} # The function to measure the quality of a split
        
rf_grid_random_search_model = mb.grid_search_cv(
    n_splits = 2, # Number of cross-validation splits
    classifier = RandomForestClassifier, # Classifier name, e.g. RandomForestClassifier
    keras_function = None, # Define Keras function. If Keras is not used, then leave this parameter blank
    grid_params = grid, # Grid space
    dev_df = data['data_{}'.format(sample_values_solution[0])],  # Development sample that this will analysis will be performed
    feats = feats_best_rf_back, # List of predictor names
    target = target_variable_name, # Target variable name
    weight_variable = weight_variable_name, # Weight variable name
    randomized_search = True, # Set to True if randomized grid search will be performed, or to False if exhaustive grid search will be performed
    n_random_grids = 100, # Number of grid searches when randomized_search=True. If randomized_search=False, then this parameter is not applicable
    random_state = None, # If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator; If None, the random number generator is the RandomState instance used by np.random.
    n_jobs = 8 # Number of jobs to run in parallel
)

# Save the best parameters
opt_params_random = rf_grid_random_search_model.best_params_
print('The best hyperparameter set is:', opt_params_random)
print('Mean loss function for cross-validation test data: ', -rf_grid_random_search_model.cv_results_['mean_test_score'][rf_grid_random_search_model.best_index_])
print('Standard deviation loss function for cross-validation test data: ', rf_grid_random_search_model.cv_results_['std_test_score'][rf_grid_random_search_model.best_index_])
#print('Mean Gini for OOT data', 2*rf_grid_random_search_model.score(oos[feats_best_rf_back], oos[target_variable_name])-1)

rp.plot_cross_validation_score(model=rf_grid_random_search_model)

# Compute the score for the OOT data
for k in data.keys():
    data[k].loc[:, 'rf_score_random_numeric'] = rf_grid_random_search_model.predict_proba(data[k][feats_best_rf_back].values)[:, 1]
    data[k].loc[:, 'rf_score_random_binary'] = list(map(round, data[k].loc[:, 'rf_score_random_numeric']))    

### Grid search based on GridSearchCV - also uses cross-validation

In [ ]:
# 72 models in total
grid = {'n_estimators':[100, 200],  # Number of trees, 100-500 is usually sufficient /
       'max_depth':[3, 5, 7],  # Depth of the tree, 5-6 is usually default
       'max_features':[18, 21, 24],  # Number of variables randomly sampled as candidates at each split, sqrt(p) is usually the default
       'min_samples_leaf': [1, 10], # Minimum number of observations in each leaf, 10 is usually default
       'criterion': ['gini', 'entropy']} # The function to measure the quality of a split
        
rf_grid_fixed_search_model = mb.grid_search_cv(
    n_splits = 2, # Number of cross-validation splits
    classifier = RandomForestClassifier, # Classifier name, e.g. RandomForestClassifier
    keras_function = None, # Define Keras function. If Keras is not used, then leave this parameter blank
    grid_params = grid, # Grid space
    dev_df = data['data_{}'.format(sample_values_solution[0])],  # Development sample that this will analysis will be performed
    feats = feats_best_rf_back, # List of predictor names
    target = target_variable_name, # Target variable name
    weight_variable = weight_variable_name, # Weight variable name
    randomized_search = False, # Set to True if randomized grid search will be performed, or to False if exhaustive grid search will be performed
    n_random_grids = 1, # Number of grid searches when randomized_search=True. If randomized_search=False, then this parameter is not applicable
    random_state = None, # If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator; If None, the random number generator is the RandomState instance used by np.random.
    n_jobs = 8 # Number of jobs to run in parallel
)

# Save the best parameters
opt_params_fixed = rf_grid_fixed_search_model.best_params_
print('The best hyperparameter set is:', opt_params_fixed)
print('Mean loss function for cross-validation test data: ', -rf_grid_fixed_search_model.cv_results_['mean_test_score'][rf_grid_fixed_search_model.best_index_])
print('Standard deviation loss function for cross-validation test data: ', rf_grid_fixed_search_model.cv_results_['std_test_score'][rf_grid_fixed_search_model.best_index_])
#print('Mean Gini for OOT data', 2*rf_grid_fixed_search_model.score(oos[feats_best_rf_back], oos[target_variable_name], sample_weight=oos[weight_variable_name])-1)

rp.plot_cross_validation_score(model=rf_grid_fixed_search_model)

# Compute the score for the OOT data
for k in data.keys():
    data[k].loc[:, 'rf_score_fixed_numeric'] = rf_grid_fixed_search_model.predict_proba(data[k][feats_best_rf_back].values)[:, 1]
    data[k].loc[:, 'rf_score_fixed_binary'] = list(map(round, data[k].loc[:, 'rf_score_fixed_numeric']))    

### Grid search based on Optuna - also uses cross-validation

In [ ]:
rf_optuna_search_model = mb.grid_search_optuna(
    classifier = RandomForestClassifier, # Classifier name, e.g. RandomForestClassifier
    grid_params = {
        'n_estimators': ([10, 400], 'int'),  # Number of trees, 100-500 is usually sufficient /
           'max_depth': ([2, 7], 'int'),  # Depth of the tree, 5-6 is usually default
           'max_features': (["sqrt", "log2"], 'cat'),  # Number of variables randomly sampled as candidates at each split, sqrt(p) is usually the default
           'min_samples_leaf': ([0.01, 0.04], 'float'), # Minimum number of observations in each leaf, 10 is usually default
           'min_samples_split': ([0.01, 0.04], 'float'), # The minimum number of observations required to split an internal node
        # If int then min_samples_split is the minimum number of observations
        # If float then then minimum number of observations is a fraction and ceil(min_samples_split * n_samples) 
           'criterion': (['gini', 'entropy'], 'cat')}, # The function to measure the quality of a split
    dev_df = data['data_{}'.format(sample_values_solution[0])], # dev_df: Development sample that this will analysis will be performed
    feats = feats_best_rf_back, # List of predictor names
    target = target_variable_name, # Target variable name
    weight_variable = weight_variable_name, # Weight variable name
    random_state = 42, # If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator; If None, the random number generator is the RandomState instance used by np.random.
    n_splits_kfold = 2, # Number of cross-validation splits
    n_repeats_kfold = 1, # Number that KFold cross-validation will be repeated
    n_random_grids = 100, # Number of grid searches from Optuna. The more searches, the longer that the algorithm will take to complete. 
    timeout = 100000, # Time in seconds that Optuna will stop
    n_jobs = -1 # Number of jobs to run in parallel
    )

rf_optuna_search_model.optimize()

# Save the best parameters
opt_params_optuna = rf_optuna_search_model.best_params()
opt_loss_optuna = rf_optuna_search_model.best_score()
print('Best Hyperparameters (Optuna): ', opt_params_optuna)
print('Mean loss function for cross-validation (Optuna): ', opt_loss_optuna)

rf_optuna_search_model.train_best_model()

# Compute the score for the OOT data
for k in data.keys():
    data[k].loc[:, 'rf_score_optuna_numeric'] = rf_optuna_search_model.predict_probabilities(X_test = data[k][feats_best_rf_back])[:, 1]
    data[k].loc[:, 'rf_score_optuna_binary'] = list(map(round, data[k].loc[:, 'rf_score_optuna_numeric']))

## Calculate RF feature importance

In [ ]:
# Load RF machine leanring library
from sklearn.ensemble import RandomForestClassifier
import joblib

# Define the random forest model
opt_params = opt_params_optuna
rf = RandomForestClassifier(n_estimators=[x[1] for x in list(opt_params.items()) if x[0]=='n_estimators'][0], 
                            max_depth=[x[1] for x in list(opt_params.items()) if x[0]=='max_depth'][0], 
                            max_features=[x[1] for x in list(opt_params.items()) if x[0]=='max_features'][0], 
                            min_samples_leaf=[x[1] for x in list(opt_params.items()) if x[0]=='min_samples_leaf'][0], 
                            min_samples_split=[x[1] for x in list(opt_params.items()) if x[0]=='min_samples_split'][0], 
                            criterion=[x[1] for x in list(opt_params.items()) if x[0]=='criterion'][0], 
                            random_state=0)

# Train the model
rf_model = mb.fit_model_weight(data['data_{}'.format(sample_values_solution[0])], 
                               feats_best_rf_back, 
                               target_variable_name, 
                               weight_variable_name, 
                               rf, 
                               data_path + '/output/' + 'rf.pkl')

# Calculate feature importance
feat_imprtnce_dictnry = mb.feature_imp(rf_model, feats_best_rf_back, data_path, 'Feature_importance_RF.csv')
display(feat_imprtnce_dictnry)

## Produce RF reports

In [ ]:
rf_iterative_report_class = rp.binary_regression_report(
    predictions_dictionary = data, 
    target_variable = target_variable_name, 
    predicted_score_numeric = 'rf_score_iterative_numeric', 
    amount_variable_name = amount_variable_name_solution, 
    weight_variable_name = weight_variable_name_solution, 
    sample_values_dict = sample_values_dict, 
    select_top_percent = 100, 
    n_bands = 10, 
    rows = 10, 
    data_path = data_path
    )
rf_iterative_eval = rf_iterative_report_class.get_evaluation(predicted_score_binary = 'rf_score_iterative_binary',
                                                            filename = 'evaluation_metrics_RF_iterative.csv')

In [ ]:
rf_random_report_class = rp.binary_regression_report(
    predictions_dictionary = data, 
    target_variable = target_variable_name, 
    predicted_score_numeric = 'rf_score_random_numeric', 
    amount_variable_name = amount_variable_name_solution, 
    weight_variable_name = weight_variable_name_solution, 
    sample_values_dict = sample_values_dict, 
    select_top_percent = 100, 
    n_bands = 10, 
    rows = 10, 
    data_path = data_path
    )
rf_random_eval = rf_random_report_class.get_evaluation(predicted_score_binary = 'rf_score_random_binary',
                                                            filename = 'evaluation_metrics_RF_random.csv')

In [ ]:
rf_fixed_report_class = rp.binary_regression_report(
    predictions_dictionary = data, 
    target_variable = target_variable_name, 
    predicted_score_numeric = 'rf_score_fixed_numeric', 
    amount_variable_name = amount_variable_name_solution, 
    weight_variable_name = weight_variable_name_solution, 
    sample_values_dict = sample_values_dict, 
    select_top_percent = 100, 
    n_bands = 10, 
    rows = 10, 
    data_path = data_path
    )
rf_fixed_eval = rf_fixed_report_class.get_evaluation(predicted_score_binary = 'rf_score_fixed_binary',
                                                            filename = 'evaluation_metrics_RF_fixed.csv')

In [ ]:
rf_optuna_report_class = rp.binary_regression_report(
    predictions_dictionary = data, 
    target_variable = target_variable_name, 
    predicted_score_numeric = 'rf_score_optuna_numeric', 
    amount_variable_name = amount_variable_name_solution, 
    weight_variable_name = weight_variable_name_solution, 
    sample_values_dict = sample_values_dict, 
    select_top_percent = 100, 
    n_bands = 10, 
    rows = 10, 
    data_path = data_path
    )
rf_optuna_eval = rf_optuna_report_class.get_evaluation(predicted_score_binary = 'rf_score_optuna_binary',
                                                            filename = 'evaluation_metrics_RF_optuna.csv')

## Calculate RF Lifting table

In [ ]:
binary_report_class = rf_optuna_report_class

In [ ]:
# Create Lift table
binary_report_lift_table = binary_report_class.create_lift_table(filename = 'lift_table_RF_')

## Plots RF

### Plot RF Detection rate vs. Population Distribution

In [ ]:
# Create folder, if it doesn't exist
folder_name = 'graphs_RF'
ufun.create_folder(data_path = data_path, 
                   folder_name = 'output/{}'.format(folder_name))

In [ ]:
binary_report_class.plot_ADR_Quantile(
        folder_name = folder_name,
        xlim=None, 
        ylim=None
        )

### Plot RF Cum. Detection rate vs. Population Distribution (Gains chart)

In [ ]:
binary_report_class.plot_cADR_Quantile(
        folder_name = folder_name,
        xlim=None, 
        ylim=None
        )

### Plot RF FPR vs. Population Distribution

In [ ]:
binary_report_class.plot_FPR_Quantile(
        folder_name = folder_name,
        xlim=None, 
        ylim=None
        )

### Plot RF Cum. FPR vs. Population Distribution

In [ ]:
binary_report_class.plot_cFPR_Quantile(
        folder_name = folder_name,
        xlim=None, 
        ylim=None
        )

### Plot RF ROC curve

In [ ]:
binary_report_class.plot_ROC_curve(folder_name = folder_name)

### Plot RF Precision Recall curve

In [ ]:
binary_report_class.plot_precision_recall_curve(folder_name = folder_name)

### Plot RF F1 score, Accuracy, Sensitivity, Specificity, Precision vs Cutoff

In [ ]:
binary_report_class.plot_cutoffs(
        folder_name = folder_name,
        n_bands = 100, # Number of bands between 0 and 1
        return_table=False # Set to True in order to return the table that produced the graph, otherwise set to False
        )

# Random Forest with H2O

In [ ]:
# Load the data
import h2o 
h2o.init(nthreads=8, max_mem_size=12)

data_h2o = data.copy()
for k in data.keys():
    # load pandas dataframe into H2O dataframe
    data_h2o[k] = h2o.H2OFrame(data[k])
    # Define response variable to be categorical for classification model
    data_h2o[k][target_variable_name] = data_h2o[k][target_variable_name].asfactor()

## Feature selection RF H2O

In [ ]:
# Provide input data to feature selection class
select = fe.SelectBest_weight(df=data['data_{}'.format(sample_values_solution[0])], 
                              target=target_variable_name, 
                              weight=weight_variable_name)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=200, max_depth=5, random_state=1234, n_jobs=6)
rf_importance, feats_best_rf = select.top_rf_feat(feats=keep_num_vars_gini, model=model_rf, n=rf_initial_screening_var_threshold)
print('The top Random Forest features are: \n', feats_best_rf)
rf_importance							  

In [ ]:
feats_best_rf_back = select.backward_recur(feats=feats_best_rf, 
                                           oos=data['data_{}'.format(sample_values_solution[1])], 
                                           model=model_rf, 
                                           min_feats=rf_backward_selection_var_threshold, 
                                           classification=True)
print(feats_best_rf_back)

## Grid search and train RF machine learning model

### Grid search based on iterative step

In [ ]:
# Hyperparameter tuning: The process works as follows:
# 1) Iterate n_estimators (grid) for the fixed values of max_depth and max_features (params). Select the value of n_estimators that minimizes the loss function. 
# 2) Keep the n_estimators value from the previous step. Iterate max_depth (grid) for the fixed value of max_features (params). Select the value of max_depth that minimizes the loss function. 
# 3) Keep the max_depth value from the previous step. Iterate max_features (grid). Select the value of max_features that minimizes the loss function. 
import math
# 18 models in total
grid = {'n_estimators':[10, 50, 80, 100, 200, 400],  # Number of trees, 100-500 is usually sufficient /
       'max_depth':[2, 5, 10, 20],  # Depth of the tree, 5-6 is usually default
       'max_features':[*range(1,len(feats_best_rf_back),math.ceil(len(feats_best_rf_back)/5))],  # Number of variables randomly sampled as candidates at each split, sqrt(p) is usually the default
       'min_samples_leaf': [1, 10, 100]} # Minimum number of observations in each leaf, 10 is usually default

params = {'n_estimators': 100, 'max_depth':10, 'max_features':math.ceil(math.sqrt(len(feats_best_rf_back))), 'min_samples_leaf':10}

opt_params_iterative, loss = mb.step_search_weight(estimator=RandomForestClassifier, params=params, grid=grid, target=target_variable_name, 
                                  weight=weight_variable_name, dev=data['data_{}'.format(sample_values_solution[0])], val=data['data_{}'.format(sample_values_solution[1])], keep=feats_best_rf_back)
print('\n Best Parameters')
print(opt_params_iterative)
print(loss)

# Load RF machine leanring library
from sklearn.ensemble import RandomForestClassifier
import joblib

# Define the random forest model
rf = RandomForestClassifier(n_estimators=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='n_estimators'][0], 
                            max_depth=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='max_depth'][0], 
                            max_features=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='max_features'][0], 
                            min_samples_leaf=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='min_samples_leaf'][0], 
                            random_state=0)

# Train the model
rf_model = mb.fit_model_weight(df=data['data_{}'.format(sample_values_solution[0])], 
                               feats=feats_best_rf_back, 
                               target=target_variable_name, 
                               weight=weight_variable_name, 
                               model=rf, 
                               model_name=data_path + '/output/' + 'rf_h2o_iterative.pkl')

# Compute the score for the OOT data
for k in data.keys():
    data[k].loc[:, 'rf_score_iterative_numeric'] = rf_model.predict_proba(data[k].loc[:, feats_best_rf_back].values)[:, 1]
    data[k].loc[:, 'rf_score_iterative_binary'] = list(map(round, data[k].loc[:, 'rf_score_iterative_numeric']))    

### Grid search based on H2O random grid space

In [ ]:
import timeit
start = timeit.default_timer()

# Hyperparameter tuning using H2O random grid space: - NOTE: THIS TAKES 5 MINUTES TO RUN!
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators import H2ORandomForestEstimator
import math

# RF hyperparameters
# 1,440 models in total
grid_h2o_random = {'ntrees': [i*50 for i in range(1, 10)], # Number of trees, 100-500 is usually sufficient
                'max_depth': list(range(2, 20, 3)), # Depth of the tree, 5-6 is usually default
                'mtries': [i*2 for i in range(1, math.ceil(len(feats_best_rf_back)/2))], # Number of variables randomly sampled as candidates at each split, sqrt(p) is usually the default
                'min_rows': [1, 10, 100], # Minimum number of observations in each leaf, 10 is usually default
                'nbins': [20, 30], #For numerical columns (real/int), build a histogram of (at least) this many bins, then split at the best point, 30 is usually default
                'nbins_cats': [10, 15], # For categorical columns (factors), build a histogram of this many bins, then split at the best point. Higher values can lead to more overfitting, 10-20 is usally default
                'sample_rate': [0.63], # Row sample rate per tree (from 0.0 to 1.0), 0.63 is usually default
                'col_sample_rate_per_tree': [1]} # Column sample rate per tree (from 0.0 to 1.0), 1 is usually default

# Search criteria
search_criteria = {'strategy': 'RandomDiscrete', 'max_models': 10, 'seed': 1}

# Train and validate a random grid of RFs
rf_h2o_grid_random = H2OGridSearch(model=H2ORandomForestEstimator,
                          grid_id='rf_h2o_grid_random',
                          hyper_params=grid_h2o_random,
                          search_criteria=search_criteria)

rf_h2o_grid_random.train(x=feats_best_rf_back, 
                         y=target_variable_name, 
                         training_frame=data_h2o['data_{}'.format(sample_values_solution[0])], 
#                         validation_frame=data_h2o['data_{}'.format(sample_values_solution[1])], 
                         weights_column=weight_variable_name, 
                         seed=1, 
                         nfolds=2)

stop = timeit.default_timer()
print('Execution time in seconds: ', stop - start)  
# Execution time in seconds:  53.19511840000001

# Get the grid results, sorted by validation logloss
rf_h2o_grid_random_perf = rf_h2o_grid_random.get_grid(sort_by='logloss', decreasing=False)
print(rf_h2o_grid_random_perf)

In [ ]:
# Build random forest model in H2O, based on the H2O random grid search. Grab the best model.
rf_h2o_random = rf_h2o_grid_random_perf.models[0]

# Check the random forest model performance on the OOT sample
rf_h2o_model_performance = rf_h2o_random.model_performance(data_h2o['data_{}'.format(sample_values_solution[1])])
print(rf_h2o_model_performance)
print('Gini for the OOT sample is: ', 2*rf_h2o_model_performance.auc()-1)

# Check the variable importance 
var_imp=[(x[0],x[2]) for x in rf_h2o_random.varimp()]
for v1, v2 in var_imp:
    print (v1,v2)

# Compute the score for the OOT data
for k in data_h2o.keys():
    data_h2o[k]['rf_h2o_score_random_numeric'] = rf_h2o_random.predict(data_h2o[k])[2]
    data_h2o[k]['rf_h2o_score_random_binary'] = data_h2o[k]['rf_h2o_score_random_numeric'].round(digits=0)

### Grid search based on H2O fixed grid space

In [ ]:
# Hyperparameter tuning using H2O fixed grid space: - NOTE: THIS TAKES SEVERAL MINUTES TO RUN!
import timeit
start = timeit.default_timer()

from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators import H2ORandomForestEstimator
import math

# 27 models in total
grid_h2o_fixed = {'ntrees':[50, 100, 200],  # Number of trees, 100-500 is usually sufficient
       'max_depth':[8, 10, 12],  # Depth of the tree, 5-6 is usually default
       'mtries':[8, 10, 12],  # Number of variables randomly sampled as candidates at each split, sqrt(p) is usually the default
        'min_rows': [10],  # Minimum number of observations in each leaf, 10 is usually default
        'nbins': [30],  #For numerical columns (real/int), build a histogram of (at least) this many bins, then split at the best point, 30 is usually default
        'nbins_cats': [15],  # For categorical columns (factors), build a histogram of this many bins, then split at the best point. Higher values can lead to more overfitting, 10-20 is usally default
        'sample_rate': [0.63],  # Row sample rate per tree (from 0.0 to 1.0), 0.63 is usually default
        'col_sample_rate_per_tree': [1]} # Column sample rate per tree (from 0.0 to 1.0), 1 is usually default

# Train and validate a cartesian grid of RFs
rf_h2o_grid_fixed = H2OGridSearch(model=H2ORandomForestEstimator,
                          grid_id='rf_h2o_grid_fixed',
                          hyper_params=grid_h2o_fixed)

rf_h2o_grid_fixed.train(x=feats_best_rf_back, 
                        y=target_variable_name, 
                        training_frame=data_h2o['data_{}'.format(sample_values_solution[0])], 
#                        validation_frame=data_h2o['data_{}'.format(sample_values_solution[1])], 
                        weights_column=weight_variable_name, 
                        seed=1, 
                        nfolds=2)

stop = timeit.default_timer()
print('Execution time in seconds: ', stop - start)  
# Execution time in seconds:  93.57661189999999

# Get the grid results, sorted by validation logloss
rf_h2o_grid_fixed_perf = rf_h2o_grid_fixed.get_grid(sort_by='logloss', decreasing=False)
print(rf_h2o_grid_fixed_perf)

In [ ]:
# Build fixed forest model in H2O, based on the H2O fixed grid search. Grab the best model.
rf_h2o_fixed = rf_h2o_grid_fixed_perf.models[0]

# Check the random forest model performance on the OOT sample
rf_h2o_model_performance = rf_h2o_fixed.model_performance(data_h2o['data_{}'.format(sample_values_solution[1])])
print(rf_h2o_model_performance)
print('Gini for the OOT sample is: ', 2*rf_h2o_model_performance.auc()-1)

# Check the variable importance 
var_imp=[(x[0],x[2]) for x in rf_h2o_fixed.varimp()]
for v1, v2 in var_imp:
    print (v1,v2)

# Compute the score for the OOT data
for k in data_h2o.keys():
    data_h2o[k]['rf_h2o_score_fixed_numeric'] = rf_h2o_fixed.predict(data_h2o[k])[2]
    data_h2o[k]['rf_h2o_score_fixed_binary'] = data_h2o[k]['rf_h2o_score_fixed_numeric'].round(digits=0)

In [ ]:
# Convert H2O dataframe to pandas dataframe
for k in data_h2o.keys():
    data[k]=data_h2o[k].as_data_frame(use_pandas=True, use_multi_thread=True)

## Produce H2O RF reports

In [ ]:
rf_h2o_random_report_class = rp.binary_regression_report(
    predictions_dictionary = data, 
    target_variable = target_variable_name, 
    predicted_score_numeric = 'rf_h2o_score_random_numeric', 
    amount_variable_name = amount_variable_name_solution, 
    weight_variable_name = weight_variable_name_solution, 
    sample_values_dict = sample_values_dict, 
    select_top_percent = 100, 
    n_bands = 10, 
    rows = 10, 
    data_path = data_path
    )
rf_h2o_random_eval = rf_h2o_random_report_class.get_evaluation(predicted_score_binary = 'rf_h2o_score_random_binary', 
                                                       filename = 'evaluation_metrics_RF_H2O_random.csv')

In [ ]:
rf_h2o_fixed_report_class = rp.binary_regression_report(
    predictions_dictionary = data, 
    target_variable = target_variable_name, 
    predicted_score_numeric = 'rf_h2o_score_fixed_numeric', 
    amount_variable_name = amount_variable_name_solution, 
    weight_variable_name = weight_variable_name_solution, 
    sample_values_dict = sample_values_dict, 
    select_top_percent = 100, 
    n_bands = 10, 
    rows = 10, 
    data_path = data_path
    )
rf_h2o_fixed_eval = rf_h2o_fixed_report_class.get_evaluation(predicted_score_binary = 'rf_h2o_score_fixed_binary', 
                                                       filename = 'evaluation_metrics_RF_H2O_fixed.csv')

## Calculate H2O RF Lifting table

In [ ]:
binary_report_class_h2o = rf_h2o_random_report_class

In [ ]:
# Create Lift table
binary_report_lift_table_h2o = binary_report_class_h2o.create_lift_table(filename = 'lift_table_RF_H2O_')

## Plots RF H2O

### Plot RF H2O Detection rate vs. Population Distribution

In [ ]:
# Create folder, if it doesn't exist
folder_name = 'graphs_RF_H2O'
ufun.create_folder(data_path = data_path, 
                   folder_name = 'output/{}'.format(folder_name))

In [ ]:
binary_report_class_h2o.plot_ADR_Quantile(
        folder_name = folder_name,
        xlim=None, 
        ylim=None
        )

### Plot RF H2O Cum. Detection rate vs. Population Distribution (Gains chart)

In [ ]:
binary_report_class_h2o.plot_cADR_Quantile(
        folder_name = folder_name,
        xlim=None, 
        ylim=None
        )

### Plot RF H2O FPR vs. Population Distribution

In [ ]:
binary_report_class_h2o.plot_FPR_Quantile(
        folder_name = folder_name,
        xlim=None, 
        ylim=None
        )

### Plot RF H2O Cum. FPR vs. Population Distribution

In [ ]:
binary_report_class_h2o.plot_cFPR_Quantile(
        folder_name = folder_name,
        xlim=None, 
        ylim=None
        )

### Plot RF H2O ROC curve

In [ ]:
binary_report_class_h2o.plot_ROC_curve(folder_name = folder_name)

### Plot RF H2O Precision Recall curve

In [ ]:
binary_report_class_h2o.plot_precision_recall_curve(folder_name = folder_name)

### Plot RF H2O F1 score, Accuracy, Sensitivity, Specificity, Precision vs Cutoff

In [ ]:
binary_report_class_h2o.plot_cutoffs(
        folder_name = folder_name,
        n_bands = 100, # Number of bands between 0 and 1
        return_table=False # Set to True in order to return the table that produced the graph, otherwise set to False
        )

## Save model and export MOJO file

In [ ]:
rf_h2o_model_to_save = rf_h2o_random

In [ ]:
# Save already trained model for future use
model_path=h2o.save_model(model=rf_h2o_model_to_save, path='{}/output/rf_h2o_grid_model'.format(data_path), force=True)

In [ ]:
# Save mojo file, which helps deploy the train model
rf_h2o_model_to_save.download_mojo(path='{}/output/model.zip'.format(data_path))

In [ ]:
h2o.cluster().shutdown()

# Gradient Boosting Machine

## Feature selection GBM

In [ ]:
# Provide input data to feature selection class
select = fe.SelectBest_weight(df=data['data_{}'.format(sample_values_solution[0])], 
                              target=target_variable_name, 
                              weight=weight_variable_name)

In [ ]:
# Select top features to apply a first screening
from sklearn.ensemble import GradientBoostingClassifier
model_gbm = GradientBoostingClassifier(n_estimators=200, max_depth=5, learning_rate=0.1, random_state=1234)
gbm_importance, feats_best_gbm = select.top_gbm_feat(feats=keep_num_vars_gini, model=model_gbm, n=15)
print('The top GBM features are: \n', feats_best_gbm)
gbm_importance

In [ ]:
# Backward selection with gradient boosting machine model: In each step a feature is selected to remove - this removal maximizes Gini
feats_best_gbm_back = select.backward_recur(feats=feats_best_gbm, 
                                            oos=data['data_{}'.format(sample_values_solution[1])], 
                                            model=model_gbm, 
                                            min_feats=10, 
                                            classification=True)
print(feats_best_gbm_back)

## Grid search and train GBM machine learning model

### Grid search based on iterative step search

In [ ]:
# Hyperparameter tuning: The process works as follows:
# 1) Iterate n_estimators (grid) for the fixed values of max_depth, max_features, and learning_rate (params). Select the value of n_estimators that minimizes the loss function. 
# 2) Keep the n_estimators value from the previous step. Iterate max_depth (grid) for the fixed value of max_features, learning_rate (params). Select the value of max_depth that minimizes the loss function. 
# 3) Keep the max_depth value from the previous step. Iterate max_features (grid) for the fixed value of learning_rate (params). Select the value of max_features that minimizes the loss function. 
# 4) Keep the max_features value from the previous step. Iterate learning_rate (grid). Select the value of learning_rate that minimizes the loss function. 
# 40 models in total
import numpy as np

grid = {'n_estimators':[10, 20, 50, 100, 200], # Number of trees, 100-500 is usually sufficient
       'max_depth':[2, 5, 10], # Depth of the tree, 5-6 is usually default
        "max_features":["log2","sqrt"], # Number of variables randomly sampled as candidates at each split, sqrt(p) is usually the default
#       'max_features':[*range(1,len(feats_best_gbm_back),math.ceil(len(feats_best_gbm_back)/5))], \
       'learning_rate':[0.05, 0.1, 0.2, 0.5], # Learing rate. Shrinkage is used for reducing, or shrinking, the impact of each additional fitted 
# base-learner (tree). It reduces the size of incremental steps and thus penalizes the importance of each consecutive iteration. 
# The intuition behind this technique is that it is better to improve a model by taking many small steps than by taking fewer large steps. 
# If one of the boosting iterations turns out to be erroneous, its negative impact can be easily corrected in subsequent steps. 
# High learn rates and especially values close to 1.0 typically result in overfit models with poor performance. 
# Values much smaller than .01 significantly slow down the learning process and might be reserved for overnight runs. 0.10 default
        "min_samples_leaf": np.linspace(0.01, 0.4, 10), # Minimum number of observations in each leaf, 10 is usually default
        "min_samples_split": np.linspace(0.01, 0.4, 10), # The minimum number of observations required to split an internal node
        # If int then min_samples_split is the minimum number of observations
        # If float then then minimum number of observations is a fraction and ceil(min_samples_split * n_samples) 
        "subsample":[0.5, 0.7, 0.8, 0.9, 0.95, 1.0], # The fraction of samples from n_estimators
        "criterion": ["friedman_mse",  "squared_error"] # The function to measure the quality of a split
#        , "loss":["deviance"]      
       }

params = {'n_estimators': 100, 'max_depth':4, 'max_features':"sqrt", 'learning_rate':0.1, "min_samples_leaf":0.2, "min_samples_split": 0.2, \
         "subsample":0.8, "criterion":"friedman_mse"
         # , "loss":["deviance"]
         }            

opt_params_iterative, loss = mb.step_search_weight(estimator=GradientBoostingClassifier, 
                                                   params=params, 
                                                   grid=grid, 
                                                   target=target_variable_name, 
                                                   weight=weight_variable_name, 
                                                   dev=data['data_{}'.format(sample_values_solution[0])], 
                                                   val=data['data_{}'.format(sample_values_solution[1])], 
                                                   keep=feats_best_gbm_back)
print('\n Best Parameters')
print(opt_params_iterative)
print(loss)

# Load GBM algorithm
from sklearn.ensemble import GradientBoostingClassifier

# Define the GBM model
gbm = GradientBoostingClassifier(n_estimators=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='n_estimators'][0], 
                                 max_depth=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='max_depth'][0], 
                                 max_features=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='max_features'][0], 
                                 learning_rate=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='learning_rate'][0], 
                                 min_samples_leaf=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='min_samples_leaf'][0], 
                                 min_samples_split=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='min_samples_split'][0], 
                                 subsample=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='subsample'][0], 
                                 criterion=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='criterion'][0],                                 
                                 random_state=0)

# Train the model
gbm_model = mb.fit_model_weight(data['data_{}'.format(sample_values_solution[0])], 
                                feats_best_gbm_back, 
                                target_variable_name, 
                                weight_variable_name, 
                                gbm, 
                                data_path + '/output/' + 'gbm_iterative.pkl')

# Compute the score for the OOT data
# Compute the score for the OOT data
for k in data.keys():
    data[k].loc[:, 'gbm_score_iterative_numeric'] = gbm_model.predict_proba(data[k].loc[:, feats_best_gbm_back].values)[:, 1]
    data[k].loc[:, 'gbm_score_iterative_binary'] = list(map(round, data[k].loc[:, 'gbm_score_iterative_numeric']))    

### Grid search based on RandomizedSearchCV - also uses cross-validation

In [ ]:
# 720 models in total
grid = {'n_estimators':[100, 200, 300, 400, 500], # Number of trees, 100-500 is usually sufficient
       'max_depth':[1, 2, 4, 6], # Depth of the tree, 5-6 is usually default
        "max_features":["sqrt", 'log2'], # Number of variables randomly sampled as candidates at each split, sqrt(p) is usually the default
#       'max_features':[*range(1,len(feats_best_gbm_back),math.ceil(len(feats_best_gbm_back)/5))], \
       'learning_rate':[0.1, 0.2, 0.3], # Learing rate. Shrinkage is used for reducing, or shrinking, the impact of each additional fitted 
# base-learner (tree). It reduces the size of incremental steps and thus penalizes the importance of each consecutive iteration. 
# The intuition behind this technique is that it is better to improve a model by taking many small steps than by taking fewer large steps. 
# If one of the boosting iterations turns out to be erroneous, its negative impact can be easily corrected in subsequent steps. 
# High learn rates and especially values close to 1.0 typically result in overfit models with poor performance. 
# Values much smaller than .01 significantly slow down the learning process and might be reserved for overnight runs. 0.10 default
        "min_samples_leaf": [0.01, 0.05, 0.10, 0.15], # Minimum number of observations in each leaf, 10 is usually default
        "min_samples_split": [0.1, 0.2, 0.3], # The minimum number of observations required to split an internal node
        # If int then min_samples_split is the minimum number of observations
        # If float then then minimum number of observations is a fraction and ceil(min_samples_split * n_samples) 
        "subsample":[0.8], # The fraction of samples from n_estimators
        "criterion": ["friedman_mse"] # The function to measure the quality of a split
#        , "loss":["deviance"]      
       }

gbm_grid_random_search_model = mb.grid_search_cv(
    n_splits = 2, # Number of cross-validation splits
    classifier = GradientBoostingClassifier, # Classifier name, e.g. RandomForestClassifier
    keras_function = None, # Define Keras function. If Keras is not used, then leave this parameter blank
    grid_params = grid, # Grid space
    dev_df = data['data_{}'.format(sample_values_solution[0])],  # Development sample that this will analysis will be performed
    feats = feats_best_gbm_back, # List of predictor names
    target = target_variable_name, # Target variable name
    weight_variable = weight_variable_name, # Weight variable name
    randomized_search = True, # Set to True if randomized grid search will be performed, or to False if exhaustive grid search will be performed
    n_random_grids = 100, # Number of grid searches when randomized_search=True. If randomized_search=False, then this parameter is not applicable
    random_state = None, # If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator; If None, the random number generator is the RandomState instance used by np.random.
    n_jobs = 8 # Number of jobs to run in parallel
)

# Save the best parameters
opt_params_random = gbm_grid_random_search_model.best_params_
print('The best hyperparameter set is:', opt_params_random)
print('Mean loss function for cross-validation test data: ', -gbm_grid_random_search_model.cv_results_['mean_test_score'][gbm_grid_random_search_model.best_index_])
print('Standard deviation loss function for cross-validation test data: ', gbm_grid_random_search_model.cv_results_['std_test_score'][gbm_grid_random_search_model.best_index_])
#print('Mean Gini for OOT data', 2*gbm_grid_random_search_model.score(oos[feats_best_gbm_back], oos[target_variable_name], sample_weight=oos[weight_variable_name])-1)

rp.plot_cross_validation_score(model=gbm_grid_random_search_model)

# Compute the score for the OOT data
for k in data.keys():
    data[k].loc[:, 'gbm_score_random_numeric'] = rf_grid_random_search_model.predict_proba(data[k][feats_best_rf_back].values)[:, 1]
    data[k].loc[:, 'gbm_score_random_binary'] = list(map(round, data[k].loc[:, 'gbm_score_random_numeric']))    

### Grid search based on GridSearchCV - also uses cross-validation

In [ ]:
from sklearn.model_selection import GridSearchCV

# 81 models in total
grid = {'n_estimators':[100, 200, 300], # Number of trees, 100-500 is usually sufficient
       'max_depth':[2, 4, 6], # Depth of the tree, 5-6 is usually default
        "max_features":["sqrt"], # Number of variables randomly sampled as candidates at each split, sqrt(p) is usually the default
#       'max_features':[*range(1,len(feats_best_gbm_back),math.ceil(len(feats_best_gbm_back)/5))], \
       'learning_rate':[0.1, 0.2, 0.3], # Learing rate. Shrinkage is used for reducing, or shrinking, the impact of each additional fitted 
# base-learner (tree). It reduces the size of incremental steps and thus penalizes the importance of each consecutive iteration. 
# The intuition behind this technique is that it is better to improve a model by taking many small steps than by taking fewer large steps. 
# If one of the boosting iterations turns out to be erroneous, its negative impact can be easily corrected in subsequent steps. 
# High learn rates and especially values close to 1.0 typically result in overfit models with poor performance. 
# Values much smaller than .01 significantly slow down the learning process and might be reserved for overnight runs. 0.10 default
        "min_samples_leaf": [0.05, 0.10, 0.15], # Minimum number of observations in each leaf, 10 is usually default
        "min_samples_split": [0.2], # The minimum number of observations required to split an internal node
        # If int then min_samples_split is the minimum number of observations
        # If float then then minimum number of observations is a fraction and ceil(min_samples_split * n_samples) 
        "subsample":[0.8], # The fraction of samples from n_estimators
        "criterion": ["friedman_mse"] # The function to measure the quality of a split
#        , "loss":["deviance"]      
       }

gbm_grid_fixed_search_model = mb.grid_search_cv(
    n_splits = 2, # Number of cross-validation splits
    classifier = GradientBoostingClassifier, # Classifier name, e.g. RandomForestClassifier
    keras_function = None, # Define Keras function. If Keras is not used, then leave this parameter blank
    grid_params = grid, # Grid space
    dev_df = data['data_{}'.format(sample_values_solution[0])],  # Development sample that this will analysis will be performed
    feats = feats_best_gbm_back, # List of predictor names
    target = target_variable_name, # Target variable name
    weight_variable = weight_variable_name, # Weight variable name
    randomized_search = False, # Set to True if randomized grid search will be performed, or to False if exhaustive grid search will be performed
    n_random_grids = 1, # Number of grid searches when randomized_search=True. If randomized_search=False, then this parameter is not applicable
    random_state = None, # If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator; If None, the random number generator is the RandomState instance used by np.random.
    n_jobs = 8 # Number of jobs to run in parallel
)

# Save the best parameters
opt_params_fixed = gbm_grid_fixed_search_model.best_params_
print('The best hyperparameter set is:', opt_params_fixed)
print('Mean loss function for cross-validation test data: ', -gbm_grid_fixed_search_model.cv_results_['mean_test_score'][gbm_grid_fixed_search_model.best_index_])
print('Standard deviation loss function for cross-validation test data: ', gbm_grid_fixed_search_model.cv_results_['std_test_score'][gbm_grid_fixed_search_model.best_index_])
#print('Mean Gini for OOT data', 2*gbm_grid_fixed_search_model.score(oos[feats_best_gbm_back], oos[target_variable_name], sample_weight=oos[weight_variable_name])-1)

rp.plot_cross_validation_score(model=gbm_grid_fixed_search_model)

# Compute the score for the OOT data
for k in data.keys():
    data[k].loc[:, 'gbm_score_fixed_numeric'] = rf_grid_fixed_search_model.predict_proba(data[k][feats_best_rf_back].values)[:, 1]
    data[k].loc[:, 'gbm_score_fixed_binary'] = list(map(round, data[k].loc[:, 'gbm_score_fixed_numeric']))    

### Grid search based on Optuna - also uses cross-validation

In [ ]:
gbm_optuna_search_model = mb.grid_search_optuna(
    classifier = GradientBoostingClassifier, # Classifier name, e.g. RandomForestClassifier
    grid_params = {
               'n_estimators': ([10, 500], 'int'),  # Number of trees, 100-500 is usually sufficient 
               'max_depth': ([2, 10], 'int'),  # Depth of the tree, 5-6 is usually default
               'max_features': (["sqrt", "log2"], 'cat'),  # Number of variables randomly sampled as candidates at each split, sqrt(p) is usually the default
               'learning_rate': ([0.05, 0.5], 'float'), # Learing rate. Shrinkage is used for reducing, or shrinking, the impact of each additional fitted 
                # base-learner (tree). It reduces the size of incremental steps and thus penalizes the importance of each consecutive iteration. 
                # The intuition behind this technique is that it is better to improve a model by taking many small steps than by taking fewer large steps. 
                # If one of the boosting iterations turns out to be erroneous, its negative impact can be easily corrected in subsequent steps. 
                # High learn rates and especially values close to 1.0 typically result in overfit models with poor performance. 
                # Values much smaller than .01 significantly slow down the learning process and might be reserved for overnight runs. 0.10 default
               'min_samples_leaf': ([0.01, 0.4], 'float'), # Minimum number of observations in each leaf, 10 is usually default
               'min_samples_split': ([0.01, 0.4], 'float'), # The minimum number of observations required to split an internal node
            # If int then min_samples_split is the minimum number of observations
            # If float then then minimum number of observations is a fraction and ceil(min_samples_split * n_samples) 
               'subsample': ([0.5, 1.0], 'float'), # The fraction of samples from n_estimators
               'criterion': (["friedman_mse",  "squared_error"], 'cat')}, # The function to measure the quality of a split
    #        , "loss":["deviance"]      
    dev_df = data['data_{}'.format(sample_values_solution[0])], # dev_df: Development sample that this will analysis will be performed
    feats = feats_best_gbm_back, # List of predictor names
    target = target_variable_name, # Target variable name
    weight_variable = weight_variable_name, # Weight variable name
    random_state = 42, # If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator; If None, the random number generator is the RandomState instance used by np.random.
    n_splits_kfold = 2, # Number of cross-validation splits
    n_repeats_kfold = 1, # Number that KFold cross-validation will be repeated
    n_random_grids = 100, # Number of grid searches from Optuna. The more searches, the longer that the algorithm will take to complete. 
    timeout = 100000, # Time in seconds that Optuna will stop
    n_jobs = -1 # Number of jobs to run in parallel
    )

gbm_optuna_search_model.optimize()

# Save the best parameters
opt_params_optuna = gbm_optuna_search_model.best_params()
opt_loss_optuna = gbm_optuna_search_model.best_score()
print('Best Hyperparameters (Optuna): ', opt_params_optuna)
print('Mean loss function for cross-validation (Optuna): ', opt_loss_optuna)

gbm_optuna_search_model.train_best_model()

# Compute the score for the OOT data
for k in data.keys():
    data[k].loc[:, 'gbm_score_optuna_numeric'] = gbm_optuna_search_model.predict_probabilities(X_test = data[k][feats_best_gbm_back])[:, 1]
    data[k].loc[:, 'gbm_score_optuna_binary'] = list(map(round, data[k].loc[:, 'gbm_score_optuna_numeric']))

## Calculate GBM feature importance

In [ ]:
# Load GBM algorithm
from sklearn.ensemble import GradientBoostingClassifier

# Define the GBM model
opt_params = opt_params_optuna
gbm = GradientBoostingClassifier(n_estimators=[x[1] for x in list(opt_params.items()) if x[0]=='n_estimators'][0], 
                                 max_depth=[x[1] for x in list(opt_params.items()) if x[0]=='max_depth'][0], 
                                 max_features=[x[1] for x in list(opt_params.items()) if x[0]=='max_features'][0], 
                                 learning_rate=[x[1] for x in list(opt_params.items()) if x[0]=='learning_rate'][0], 
                                 min_samples_leaf=[x[1] for x in list(opt_params.items()) if x[0]=='min_samples_leaf'][0], 
                                 min_samples_split=[x[1] for x in list(opt_params.items()) if x[0]=='min_samples_split'][0], 
                                 subsample=[x[1] for x in list(opt_params.items()) if x[0]=='subsample'][0], 
                                 criterion=[x[1] for x in list(opt_params.items()) if x[0]=='criterion'][0],                                 
                                 random_state=0)

# Train the model
gbm_model = mb.fit_model_weight(data['data_{}'.format(sample_values_solution[0])], 
                                feats_best_gbm_back, 
                                target_variable_name, 
                                weight_variable_name, 
                                gbm, 
                                data_path + '/output/' + 'gbm.pkl')

# Calculate feature importance
feat_imprtnce_dictnry_gbm = mb.feature_imp(gbm_model, feats_best_gbm_back, data_path, 'Feature_importance_GBM.csv')
display(feat_imprtnce_dictnry_gbm)

## Produce GBM reports

In [ ]:
gbm_iterative_report_class = rp.binary_regression_report(
    predictions_dictionary = data, 
    target_variable = target_variable_name, 
    predicted_score_numeric = 'gbm_score_iterative_numeric', 
    amount_variable_name = amount_variable_name_solution, 
    weight_variable_name = weight_variable_name_solution, 
    sample_values_dict = sample_values_dict, 
    select_top_percent = 100, 
    n_bands = 10, 
    rows = 10, 
    data_path = data_path
    )
gbm_iterative_eval = gbm_iterative_report_class.get_evaluation(predicted_score_binary = 'gbm_score_iterative_binary', 
                                                       filename = 'evaluation_metrics_GBM_iterative.csv')

In [ ]:
gbm_random_report_class = rp.binary_regression_report(
    predictions_dictionary = data, 
    target_variable = target_variable_name, 
    predicted_score_numeric = 'gbm_score_random_numeric', 
    amount_variable_name = amount_variable_name_solution, 
    weight_variable_name = weight_variable_name_solution, 
    sample_values_dict = sample_values_dict, 
    select_top_percent = 100, 
    n_bands = 10, 
    rows = 10, 
    data_path = data_path
    )
gbm_random_eval = gbm_random_report_class.get_evaluation(predicted_score_binary = 'gbm_score_random_binary', 
                                                       filename = 'evaluation_metrics_GBM_random.csv')

In [ ]:
gbm_fixed_report_class = rp.binary_regression_report(
    predictions_dictionary = data, 
    target_variable = target_variable_name, 
    predicted_score_numeric = 'gbm_score_fixed_numeric', 
    amount_variable_name = amount_variable_name_solution, 
    weight_variable_name = weight_variable_name_solution, 
    sample_values_dict = sample_values_dict, 
    select_top_percent = 100, 
    n_bands = 10, 
    rows = 10, 
    data_path = data_path
    )
gbm_fixed_eval = gbm_fixed_report_class.get_evaluation(predicted_score_binary = 'gbm_score_fixed_binary', 
                                                       filename = 'evaluation_metrics_GBM_fixed.csv')

In [ ]:
gbm_optuna_report_class = rp.binary_regression_report(
    predictions_dictionary = data, 
    target_variable = target_variable_name, 
    predicted_score_numeric = 'gbm_score_optuna_numeric', 
    amount_variable_name = amount_variable_name_solution, 
    weight_variable_name = weight_variable_name_solution, 
    sample_values_dict = sample_values_dict, 
    select_top_percent = 100, 
    n_bands = 10, 
    rows = 10, 
    data_path = data_path
    )
gbm_optuna_eval = gbm_optuna_report_class.get_evaluation(predicted_score_binary = 'gbm_score_optuna_binary', 
                                                       filename = 'evaluation_metrics_GBM_optuna.csv')

## Calculate GBM Lifting table

In [ ]:
binary_report_class = gbm_optuna_report_class

In [ ]:
# Create Lift table
binary_report_lift_table = binary_report_class.create_lift_table(filename = 'lift_table_GBM_')

## Plots GBM

### Plot GBM Detection rate vs. Population Distribution

In [ ]:
# Create folder, if it doesn't exist
folder_name = 'graphs_GBM'
ufun.create_folder(data_path = data_path, 
                   folder_name = 'output/{}'.format(folder_name))

In [ ]:
binary_report_class.plot_ADR_Quantile(
        folder_name = folder_name,
        xlim=None, 
        ylim=None
        )

### Plot GBM Cum. Detection rate vs. Population Distribution (Gains chart)

In [ ]:
binary_report_class.plot_cADR_Quantile(
        folder_name = folder_name,
        xlim=None, 
        ylim=None
        )

### Plot GBM FPR vs. Population Distribution

In [ ]:
binary_report_class.plot_FPR_Quantile(
        folder_name = folder_name,
        xlim=None, 
        ylim=None
        )

### Plot GBM Cum. FPR vs. Population Distribution

In [ ]:
binary_report_class.plot_cFPR_Quantile(
        folder_name = folder_name,
        xlim=None, 
        ylim=None
        )

### Plot GBM ROC curve

In [ ]:
binary_report_class.plot_ROC_curve(folder_name = folder_name)

### Plot GBM Precision Recall curve

In [ ]:
binary_report_class.plot_precision_recall_curve(folder_name = folder_name)

### Plot GBM F1 score, Accuracy, Sensitivity, Specificity, Precision vs Cutoff

In [ ]:
binary_report_class.plot_cutoffs(
        folder_name = folder_name,
        n_bands = 100, # Number of bands between 0 and 1
        return_table=False # Set to True in order to return the table that produced the graph, otherwise set to False
        )

# Light Gradient Boosting Machine

## Feature selection lightGBM

In [ ]:
# Provide input data to feature selection class
select = fe.SelectBest_weight(df=data['data_{}'.format(sample_values_solution[0])], 
                              target=target_variable_name, 
                              weight=weight_variable_name)

In [ ]:
# Select top features to apply a first screening
from lightgbm import LGBMClassifier
model_lgbm = LGBMClassifier(boosting_type='gbdt',  objective='binary', n_estimators=200, 
                            learning_rate=0.1, max_depth=5, random_state=1234, n_jobs=6)
lgbm_importance, feats_best_lgbm = select.top_lgbm_feat(feats=keep_num_vars_gini, model=model_lgbm, n=15)
print('The top lightGBM features are: \n', feats_best_lgbm)
lgbm_importance

In [ ]:
import timeit
start = timeit.default_timer()

# Backward selection with lightGBM model: In each step a feature is selected to remove - this removal maximizes Gini
feats_best_lgbm_back = select.backward_recur(feats=feats_best_lgbm, 
                                             oos=data['data_{}'.format(sample_values_solution[1])], 
                                             model=model_lgbm, 
                                             min_feats=10, 
                                             classification=True)
print(feats_best_lgbm_back)

stop = timeit.default_timer()
print('Execution time in seconds: ', stop - start)  
# Execution time in seconds:  11.858331700000008

## Grid search and train lightGBM machine learning model

### Grid search based on iterative step search

In [ ]:
# Hyperparameter tuning: The process works as follows:
# 1) Iterate n_estimators (grid) for the fixed values of max_depth, learning_rate, etc (params). Select the value of n_estimators that minimizes the loss function. 
# 2) Keep the n_estimators value from the previous step. Iterate max_depth (grid) for the fixed value of learning_rate, etc (params). Select the value of max_depth that minimizes the loss function. 
# 3) Keep the max_depth value from the previous step. Iterate learning_rate (grid) for the fixed value of the remaining parameters (params). Select the value of max_features that minimizes the loss function. 
# 4) ...
import math
# 38 models in total
grid = {'n_estimators':[10, 20, 50, 100, 200], # Number of trees, 100-500 is usually sufficient
       'max_depth':[2, 5, 10], # Depth of the tree, 5-6 is usually default
       'learning_rate':[0.05, 0.1, 0.2, 0.5], # Learing rate. Shrinkage is used for reducing, or shrinking, the impact of each additional fitted 
# base-learner (tree). It reduces the size of incremental steps and thus penalizes the importance of each consecutive iteration. 
# The intuition behind this technique is that it is better to improve a model by taking many small steps than by taking fewer large steps. 
# If one of the boosting iterations turns out to be erroneous, its negative impact can be easily corrected in subsequent steps. 
# High learn rates and especially values close to 1.0 typically result in overfit models with poor performance. 
# Values much smaller than .01 significantly slow down the learning process and might be reserved for overnight runs. 0.10 default
        'min_child_samples': [10, 100, 500, 1000, 2000, 5000], # Minimum number of observations in each leaf, 10 is usually default
        'subsample': [0.5, 0.7, 0.8, 0.9, 0.95, 1.0], # The fraction of samples from n_estimators
        'colsample_bytree': np.linspace(0.01, 0.4, 10), # Subsample ratio of columns when constructing each tree
        'eval_metric': ['l2', 'l1', 'logloss', 'ndcg'], # If string, it should be a built-in evaluation metric to use
        'boosting_type': ['gbdt', 'dart', 'goss'], # ‘gbdt’, traditional Gradient Boosting Decision Tree. ‘dart’, Dropouts meet Multiple Additive Regression Trees. ‘goss’, Gradient-based One-Side Sampling. ‘rf’, Random Forest
        'num_leaves': [10, 20, 30, 50], # Maximum number of tree leaves 
        'bagging_fraction': [0.5, 0.7, 0.8, 0.9, 0.95, 1.0], # The fraction of observations to be used for each iteration
        'reg_alpha': [0.1, 0.5], # L1 regularization term on weights
        'reg_lambda': [0.1, 0.5], # L2 regularization term on weights
        'lambda_l1': [0, 1, 1.5], # L1 regularization
        'lambda_l2': [0, 1], # L2 regularization
        'importance_type': ['split'], # The type of feature importance to be filled into feature_importances_. If ‘split’, result contains numbers of times the feature is used in a model. If ‘gain’, result contains total gains of splits which use the feature.
        'objective': ['binary'] # Specify the learning task and the corresponding learning objective
       }

params = {'n_estimators': 100, 'max_depth':4, 'learning_rate':0.1, 'min_child_samples': 10, 'subsample': 1.0, 
          'colsample_bytree': math.ceil(math.sqrt(len(feats_best_lgbm_back)))/len(feats_best_lgbm_back),  'metric': 'l2', 'boosting_type': 'gbdt',
          'num_leaves': 31, 'bagging_fraction':1, 'reg_alpha': 0, 'reg_lambda': 0, 'lambda_l1': 0, 'lambda_l2': 0,
          'importance_type': 'split', 'objective': 'binary'
         }            

opt_params_iterative, loss = mb.step_search_weight(
    estimator=LGBMClassifier, 
    params=params, 
    grid=grid, 
    target=target_variable_name,
    weight=weight_variable_name, 
    dev=data['data_{}'.format(sample_values_solution[0])], 
    val=data['data_{}'.format(sample_values_solution[1])], 
    keep=feats_best_lgbm_back)

print('\n Best Parameters')
print(opt_params_iterative)
print(loss)

# Define the lightGBM model
lightgbm_classifier = LGBMClassifier(n_estimators=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='n_estimators'][0], 
                                 max_depth=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='max_depth'][0], 
                                 learning_rate=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='learning_rate'][0], 
                                 min_child_samples=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='min_child_samples'][0], 
                                 subsample=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='subsample'][0], 
                                 colsample_bytree=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='colsample_bytree'][0], 
                                 metric=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='metric'][0], 
                                 boosting_type=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='boosting_type'][0], 
                                 num_leaves=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='num_leaves'][0], 
                                 bagging_fraction=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='bagging_fraction'][0], 
                                 reg_alpha=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='reg_alpha'][0], 
                                 reg_lambda=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='reg_lambda'][0], 
                                 lambda_l1=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='lambda_l1'][0], 
                                 lambda_l2=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='lambda_l2'][0], 
                                 importance_type=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='importance_type'][0], 
                                 objective=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='objective'][0], 
                                 random_state=1234, 
                                 n_jobs=6)

# Train the model
lightgbm_model = mb.fit_model_weight(data['data_{}'.format(sample_values_solution[0])], 
                                     feats_best_lgbm_back, 
                                     target_variable_name, 
                                     weight_variable_name, 
                                     lightgbm_classifier, 
                                     data_path + '/output/' + 'lgbm_iterative.pkl')

# Compute the score for the OOT data
for k in data.keys():
    data[k].loc[:, 'lightgbm_score_iterative_numeric'] = lightgbm_model.predict_proba(X = data[k][feats_best_lgbm_back])[:, 1]
    data[k].loc[:, 'lightgbm_score_iterative_binary'] = list(map(round, data[k].loc[:, 'lightgbm_score_iterative_numeric']))

### Grid search based on RandomizedSearchCV - also uses cross-validation

In [ ]:
# 1,536 models in total
grid = {'n_estimators':[50, 100, 200], # Number of trees, 100-500 is usually sufficient
       'max_depth':[2, 5], # Depth of the tree, 5-6 is usually default
       'learning_rate':[0.1, 0.2], # Learing rate. Shrinkage is used for reducing, or shrinking, the impact of each additional fitted 
# base-learner (tree). It reduces the size of incremental steps and thus penalizes the importance of each consecutive iteration. 
# The intuition behind this technique is that it is better to improve a model by taking many small steps than by taking fewer large steps. 
# If one of the boosting iterations turns out to be erroneous, its negative impact can be easily corrected in subsequent steps. 
# High learn rates and especially values close to 1.0 typically result in overfit models with poor performance. 
# Values much smaller than .01 significantly slow down the learning process and might be reserved for overnight runs. 0.10 default
        'min_child_samples': [10, 100], # Minimum number of observations in each leaf, 10 is usually default
        'subsample': [0.95, 1.0], # The fraction of samples from n_estimators
        'colsample_bytree': np.linspace(0.20, 0.23, 2), # Subsample ratio of columns when constructing each tree
        'metric': ['l2'], # If string, it should be a built-in evaluation metric to use
        'boosting_type': ['gbdt'], # ‘gbdt’, traditional Gradient Boosting Decision Tree. ‘dart’, Dropouts meet Multiple Additive Regression Trees. ‘goss’, Gradient-based One-Side Sampling. ‘rf’, Random Forest
        'num_leaves': [30, 35], # Maximum number of tree leaves 
        'bagging_fraction': [0.95, 1.0], # The fraction of observations to be used for each iteration
        'reg_alpha': [0, 0.1], # L1 regularization term on weights
        'reg_lambda': [0, 0.1], # L2 regularization term on weights
        'lambda_l1': [1], # L1 regularization
        'lambda_l2': [0], # L2 regularization
        'importance_type': ['split'], # The type of feature importance to be filled into feature_importances_. If ‘split’, result contains numbers of times the feature is used in a model. If ‘gain’, result contains total gains of splits which use the feature.
        'objective': ['binary'] # Specify the learning task and the corresponding learning objective
       }

lightgbm_grid_random_search = mb.grid_search_cv(
    n_splits = 2, # Number of cross-validation splits
    classifier = LGBMClassifier, # Classifier name, e.g. RandomForestClassifier
    keras_function = None, # Define Keras function. If Keras is not used, then leave this parameter blank
    grid_params = grid, # Grid space
    dev_df = data['data_{}'.format(sample_values_solution[0])],  # Development sample that this will analysis will be performed
    feats = feats_best_lgbm_back, # List of predictor names
    target = target_variable_name, # Target variable name
    weight_variable = weight_variable_name, # Weight variable name
    randomized_search = True, # Set to True if randomized grid search will be performed, or to False if exhaustive grid search will be performed
    n_random_grids = 100, # Number of grid searches when randomized_search=True. If randomized_search=False, then this parameter is not applicable
    random_state = None, # If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator; If None, the random number generator is the RandomState instance used by np.random.
    n_jobs = 8 # Number of jobs to run in parallel
)

# Save the best parameters
opt_params_random = lightgbm_grid_random_search.best_params_
print('The best hyperparameter set is:', opt_params_random)
print('Mean loss function for cross-validation test data: ', -lightgbm_grid_random_search.cv_results_['mean_test_score'][lightgbm_grid_random_search.best_index_])
print('Standard deviation loss function for cross-validation test data: ', lightgbm_grid_random_search.cv_results_['std_test_score'][lightgbm_grid_random_search.best_index_])
#print('Mean Gini for OOT data', 2*lightgbm_grid_random_search.score(oos[feats_best_lgbm_back], oos[target_variable_name])-1)

rp.plot_cross_validation_score(model=lightgbm_grid_random_search)

# Compute the score for the OOT data
for k in data.keys():
    data[k].loc[:, 'lightgbm_score_random_numeric'] = lightgbm_grid_random_search.predict_proba(X = data[k][feats_best_lgbm_back])[:, 1]
    data[k].loc[:, 'lightgbm_score_random_binary'] = list(map(round, data[k].loc[:, 'lightgbm_score_random_numeric']))

### Grid search based on GridSearchCV - also uses cross-validation

In [ ]:
# 1,536 models in total
grid = {'n_estimators':[50, 100, 200], # Number of trees, 100-500 is usually sufficient
       'max_depth':[2, 5], # Depth of the tree, 5-6 is usually default
       'learning_rate':[0.1, 0.2], # Learing rate. Shrinkage is used for reducing, or shrinking, the impact of each additional fitted 
# base-learner (tree). It reduces the size of incremental steps and thus penalizes the importance of each consecutive iteration. 
# The intuition behind this technique is that it is better to improve a model by taking many small steps than by taking fewer large steps. 
# If one of the boosting iterations turns out to be erroneous, its negative impact can be easily corrected in subsequent steps. 
# High learn rates and especially values close to 1.0 typically result in overfit models with poor performance. 
# Values much smaller than .01 significantly slow down the learning process and might be reserved for overnight runs. 0.10 default
        'min_child_samples': [10, 100], # Minimum number of observations in each leaf, 10 is usually default
        'subsample': [0.95, 1.0], # The fraction of samples from n_estimators
        'colsample_bytree': np.linspace(0.20, 0.23, 2), # Subsample ratio of columns when constructing each tree
        'metric': ['l2'], # If string, it should be a built-in evaluation metric to use
        'boosting_type': ['gbdt'], # ‘gbdt’, traditional Gradient Boosting Decision Tree. ‘dart’, Dropouts meet Multiple Additive Regression Trees. ‘goss’, Gradient-based One-Side Sampling. ‘rf’, Random Forest
        'num_leaves': [30, 35], # Maximum number of tree leaves 
        'bagging_fraction': [0.95, 1.0], # The fraction of observations to be used for each iteration
        'reg_alpha': [0, 0.1], # L1 regularization term on weights
        'reg_lambda': [0, 0.1], # L2 regularization term on weights
        'lambda_l1': [1], # L1 regularization
        'lambda_l2': [0], # L2 regularization
        'importance_type': ['split'], # The type of feature importance to be filled into feature_importances_. If ‘split’, result contains numbers of times the feature is used in a model. If ‘gain’, result contains total gains of splits which use the feature.
        'objective': ['binary'] # Specify the learning task and the corresponding learning objective
       }

lightgbm_grid_fixed_search = mb.grid_search_cv(
    n_splits = 2, # Number of cross-validation splits
    classifier = LGBMClassifier, # Classifier name, e.g. RandomForestClassifier
    keras_function = None, # Define Keras function. If Keras is not used, then leave this parameter blank
    grid_params = grid, # Grid space
    dev_df = data['data_{}'.format(sample_values_solution[0])],  # Development sample that this will analysis will be performed
    feats = feats_best_lgbm_back, # List of predictor names
    target = target_variable_name, # Target variable name
    weight_variable = weight_variable_name, # Weight variable name
    randomized_search = False, # Set to True if randomized grid search will be performed, or to False if exhaustive grid search will be performed
    n_random_grids = 1, # Number of grid searches when randomized_search=True. If randomized_search=False, then this parameter is not applicable
    random_state = None, # If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator; If None, the random number generator is the RandomState instance used by np.random.
    n_jobs = 8 # Number of jobs to run in parallel
)

# Save the best parameters
opt_params_fixed = lightgbm_grid_fixed_search.best_params_
print('The best hyperparameter set is:', opt_params_fixed)
print('Mean loss function for cross-validation test data: ', -lightgbm_grid_fixed_search.cv_results_['mean_test_score'][lightgbm_grid_fixed_search.best_index_])
print('Standard deviation loss function for cross-validation test data: ', lightgbm_grid_fixed_search.cv_results_['std_test_score'][lightgbm_grid_fixed_search.best_index_])
#print('Mean Gini for OOT data', 2*lightgbm_grid_fixed_search.score(oos[feats_best_lgbm_back], oos[target_variable_name], sample_weight=oos[weight_variable_name])-1)

rp.plot_cross_validation_score(model=lightgbm_grid_fixed_search)

# Compute the score for the OOT data
for k in data.keys():
    data[k].loc[:, 'lightgbm_score_fixed_numeric'] = lightgbm_grid_fixed_search.predict_proba(X = data[k][feats_best_lgbm_back])[:, 1]
    data[k].loc[:, 'lightgbm_score_fixed_binary'] = list(map(round, data[k].loc[:, 'lightgbm_score_fixed_numeric']))

### Grid search based on Optuna - also uses cross-validation

In [ ]:
lightgbm_optuna_search_model = mb.grid_search_optuna(
    classifier = LGBMClassifier, # Classifier name, e.g. RandomForestClassifier
    grid_params = {
       'n_estimators': ([10, 200], 'int'), # Number of trees, 100-500 is usually sufficient
       'max_depth': ([2, 10], 'int'), # Depth of the tree, 5-6 is usually default
       'learning_rate': ([0.05, 0.5], 'float'), # Learing rate. Shrinkage is used for reducing, or shrinking, the impact of each additional fitted 
# base-learner (tree). It reduces the size of incremental steps and thus penalizes the importance of each consecutive iteration. 
# The intuition behind this technique is that it is better to improve a model by taking many small steps than by taking fewer large steps. 
# If one of the boosting iterations turns out to be erroneous, its negative impact can be easily corrected in subsequent steps. 
# High learn rates and especially values close to 1.0 typically result in overfit models with poor performance. 
# Values much smaller than .01 significantly slow down the learning process and might be reserved for overnight runs. 0.10 default
        'min_child_samples': ([10, 5000], 'int'), # Minimum number of observations in each leaf, 10 is usually default
        'subsample': ([0.5, 1.0], 'float'), # The fraction of samples from n_estimators
        'colsample_bytree': ([0.01, 0.4], 'float'), # Subsample ratio of columns when constructing each tree
        'metric': (['l2', 'l1', 'logloss'], 'cat'), # If string, it should be a built-in evaluation metric to use
        'boosting_type': (['gbdt', 'dart', 'goss'], 'cat'), # ‘gbdt’, traditional Gradient Boosting Decision Tree. ‘dart’, Dropouts meet Multiple Additive Regression Trees. ‘goss’, Gradient-based One-Side Sampling. ‘rf’, Random Forest
        'num_leaves': ([10, 50], 'int'), # Maximum number of tree leaves 
        'bagging_fraction': ([0.5, 1.0], 'float'), # The fraction of observations to be used for each iteration
        'reg_alpha': ([0.1, 0.5], 'float'), # L1 regularization term on weights
        'reg_lambda': ([0.1, 0.5], 'float'), # L2 regularization term on weights
        'lambda_l1': ([0, 1.5], 'float'), # L1 regularization
        'lambda_l2': ([0, 1], 'float'), # L2 regularization
        'importance_type': (['split'], 'cat'), # The type of feature importance to be filled into feature_importances_. If ‘split’, result contains numbers of times the feature is used in a model. If ‘gain’, result contains total gains of splits which use the feature.
        'objective': (['binary'], 'cat') # Specify the learning task and the corresponding learning objective
    },
    dev_df = data['data_{}'.format(sample_values_solution[0])], # dev_df: Development sample that this will analysis will be performed
    feats = feats_best_lgbm_back, # List of predictor names
    target = target_variable_name, # Target variable name
    weight_variable = weight_variable_name, # Weight variable name
    random_state = 42, # If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator; If None, the random number generator is the RandomState instance used by np.random.
    n_splits_kfold = 2, # Number of cross-validation splits
    n_repeats_kfold = 1, # Number that KFold cross-validation will be repeated
    n_random_grids = 100, # Number of grid searches from Optuna. The more searches, the longer that the algorithm will take to complete. 
    timeout = 100000, # Time in seconds that Optuna will stop
    n_jobs = -1 # Number of jobs to run in parallel
    )

lightgbm_optuna_search_model.optimize()

# Save the best parameters
opt_params_optuna = lightgbm_optuna_search_model.best_params()
opt_loss_optuna = lightgbm_optuna_search_model.best_score()
print('Best Hyperparameters (Optuna): ', opt_params_optuna)
print('Mean loss function for cross-validation (Optuna): ', opt_loss_optuna)

lightgbm_optuna_search_model.train_best_model()

# Compute the score for the OOT data
for k in data.keys():
    data[k].loc[:, 'lightgbm_score_optuna_numeric'] = lightgbm_optuna_search_model.predict_probabilities(X_test = data[k][feats_best_lgbm_back])[:, 1]
    data[k].loc[:, 'lightgbm_score_optuna_binary'] = list(map(round, data[k].loc[:, 'lightgbm_score_optuna_numeric']))

## Calculate lightGBM feature importance

In [ ]:
# Define the lightGBM model
opt_params = opt_params_optuna
lightgbm_classifier = LGBMClassifier(n_estimators=[x[1] for x in list(opt_params.items()) if x[0]=='n_estimators'][0], 
                                 max_depth=[x[1] for x in list(opt_params.items()) if x[0]=='max_depth'][0], 
                                 learning_rate=[x[1] for x in list(opt_params.items()) if x[0]=='learning_rate'][0], 
                                 min_child_samples=[x[1] for x in list(opt_params.items()) if x[0]=='min_child_samples'][0], 
                                 subsample=[x[1] for x in list(opt_params.items()) if x[0]=='subsample'][0], 
                                 colsample_bytree=[x[1] for x in list(opt_params.items()) if x[0]=='colsample_bytree'][0], 
                                 metric=[x[1] for x in list(opt_params.items()) if x[0]=='metric'][0], 
                                 boosting_type=[x[1] for x in list(opt_params.items()) if x[0]=='boosting_type'][0], 
                                 num_leaves=[x[1] for x in list(opt_params.items()) if x[0]=='num_leaves'][0], 
                                 bagging_fraction=[x[1] for x in list(opt_params.items()) if x[0]=='bagging_fraction'][0], 
                                 reg_alpha=[x[1] for x in list(opt_params.items()) if x[0]=='reg_alpha'][0], 
                                 reg_lambda=[x[1] for x in list(opt_params.items()) if x[0]=='reg_lambda'][0], 
                                 lambda_l1=[x[1] for x in list(opt_params.items()) if x[0]=='lambda_l1'][0], 
                                 lambda_l2=[x[1] for x in list(opt_params.items()) if x[0]=='lambda_l2'][0], 
                                 importance_type=[x[1] for x in list(opt_params.items()) if x[0]=='importance_type'][0], 
                                 objective=[x[1] for x in list(opt_params.items()) if x[0]=='objective'][0], 
                                 random_state=1234, 
                                 n_jobs=6)

# Train the model
lightgbm_model = mb.fit_model_weight(data['data_{}'.format(sample_values_solution[0])], 
                                     feats_best_lgbm_back, 
                                     target_variable_name, 
                                     weight_variable_name, 
                                     lightgbm_classifier, 
                                     data_path + '/output/' + 'lgbm.pkl')

# Calculate feature importance
feat_imprtnce_dictnry_lgbm = mb.feature_imp(lightgbm_model, feats_best_lgbm_back, data_path, 'Feature_importance_lightGBM.csv')
display(feat_imprtnce_dictnry_lgbm)

## Produce lightGBM reports

In [ ]:
lightgbm_iterative_report_class = rp.binary_regression_report(
    predictions_dictionary = data, 
    target_variable = target_variable_name, 
    predicted_score_numeric = 'lightgbm_score_iterative_numeric', 
    amount_variable_name = amount_variable_name_solution, 
    weight_variable_name = weight_variable_name_solution, 
    sample_values_dict = sample_values_dict, 
    select_top_percent = 100, 
    n_bands = 10, 
    rows = 10, 
    data_path = data_path
    )
lightgbm_iterative_eval = lightgbm_iterative_report_class.get_evaluation(predicted_score_binary = 'lightgbm_score_iterative_binary', 
                                                       filename = 'evaluation_metrics_lightGBM_iterative.csv')

In [ ]:
lightgbm_random_report_class = rp.binary_regression_report(
    predictions_dictionary = data, 
    target_variable = target_variable_name, 
    predicted_score_numeric = 'lightgbm_score_random_numeric', 
    amount_variable_name = amount_variable_name_solution, 
    weight_variable_name = weight_variable_name_solution, 
    sample_values_dict = sample_values_dict, 
    select_top_percent = 100, 
    n_bands = 10, 
    rows = 10, 
    data_path = data_path
    )
lightgbm_random_eval = lightgbm_random_report_class.get_evaluation(predicted_score_binary = 'lightgbm_score_random_binary', 
                                                       filename = 'evaluation_metrics_lightGBM_random.csv')

In [ ]:
lightgbm_fixed_report_class = rp.binary_regression_report(
    predictions_dictionary = data, 
    target_variable = target_variable_name, 
    predicted_score_numeric = 'lightgbm_score_fixed_numeric', 
    amount_variable_name = amount_variable_name_solution, 
    weight_variable_name = weight_variable_name_solution, 
    sample_values_dict = sample_values_dict, 
    select_top_percent = 100, 
    n_bands = 10, 
    rows = 10, 
    data_path = data_path
    )
lightgbm_fixed_eval = lightgbm_fixed_report_class.get_evaluation(predicted_score_binary = 'lightgbm_score_fixed_binary', 
                                                       filename = 'evaluation_metrics_lightGBM_fixed.csv')

In [ ]:
lightgbm_optuna_report_class = rp.binary_regression_report(
    predictions_dictionary = data, 
    target_variable = target_variable_name, 
    predicted_score_numeric = 'lightgbm_score_optuna_numeric', 
    amount_variable_name = amount_variable_name_solution, 
    weight_variable_name = weight_variable_name_solution, 
    sample_values_dict = sample_values_dict, 
    select_top_percent = 100, 
    n_bands = 10, 
    rows = 10, 
    data_path = data_path
    )
lightgbm_optuna_eval = lightgbm_optuna_report_class.get_evaluation(predicted_score_binary = 'lightgbm_score_optuna_binary', 
                                                       filename = 'evaluation_metrics_lightGBM_optuna.csv')

## Calculate lightGBM Lifting table

In [ ]:
binary_report_class = lightgbm_optuna_report_class

In [ ]:
# Create Lift table
binary_report_lift_table = binary_report_class.create_lift_table(filename = 'lift_table_lightGBM_')

## Plots lightGBM

### Plot lightGBM Detection rate vs. Population Distribution

In [ ]:
# Create folder, if it doesn't exist
folder_name = 'graphs_lightGBM'
ufun.create_folder(data_path = data_path, 
                   folder_name = 'output/{}'.format(folder_name))

In [ ]:
binary_report_class.plot_ADR_Quantile(
        folder_name = folder_name,
        xlim=None, 
        ylim=None
        )

### Plot lightGBM Cum. Detection rate vs. Population Distribution (Gains chart)

In [ ]:
binary_report_class.plot_cADR_Quantile(
        folder_name = folder_name,
        xlim=None, 
        ylim=None
        )

### Plot lightGBM FPR vs. Population Distribution

In [ ]:
binary_report_class.plot_FPR_Quantile(
        folder_name = folder_name,
        xlim=None, 
        ylim=None
        )

### Plot lightGBM Cum. FPR vs. Population Distribution

In [ ]:
binary_report_class.plot_cFPR_Quantile(
        folder_name = folder_name,
        xlim=None, 
        ylim=None
        )

### Plot lightGBM ROC curve

In [ ]:
binary_report_class.plot_ROC_curve(folder_name = folder_name)

### Plot lightGBM Precision Recall curve

In [ ]:
binary_report_class.plot_precision_recall_curve(folder_name = folder_name)

### Plot lightGBM F1 score, Accuracy, Sensitivity, Specificity, Precision vs Cutoff

In [ ]:
binary_report_class.plot_cutoffs(
        folder_name = folder_name,
        n_bands = 100, # Number of bands between 0 and 1
        return_table=False # Set to True in order to return the table that produced the graph, otherwise set to False
        )

# Keras NN

## Feature selection Keras Neural Network

### First find neural network hyperparameters for a network using RandomizedSearchCV, then select the top variables based on that network

In [ ]:
import math
from keras.wrappers.scikit_learn import KerasClassifier

grid = [
# SGD optimizer
    {
    'num_neurons': [math.floor((len(keep_num_vars_gini)+1)/2)]
    , 'num_hidden_layers': [1, 2]
    , 'input_dim': [data['data_{}'.format(sample_values_solution[0])][keep_num_vars_gini].shape[1]]
    , 'kernel_initializer': ['normal', 'uniform']
    , 'activation': ['relu', 'elu', 'linear', 'softplus']
    , 'kernel_constraint': [0,1,2,3,4,5]
    , 'dropout_rate': [0, 0.25, 0.5, 0.75]
    , 'output_kernel_initializer': ['uniform']
    , 'output_activation': ['sigmoid']
    , 'loss': ['binary_crossentropy']
    , 'optimizer': ['SGD']
    , 'learning_rate': [0.01, 0.001, 0.10, 0.30]
    , 'momentum': [0.0, 0.25, 0.5]
    , 'rho': [99999]
    , 'beta_1': [99999]
    , 'beta_2': [99999]
    , 'return_metrics': ['False']
    , 'batch_size': [None, 32, 64, 128]
    , 'epochs': [1, 10, 100, 500, 1000]
},
# Adadelta optimizer
    {
    'num_neurons': [math.floor((len(keep_num_vars_gini)+1)/2)]
    , 'num_hidden_layers': [1, 2]
    , 'input_dim': [data['data_{}'.format(sample_values_solution[0])][keep_num_vars_gini].shape[1]]
    , 'kernel_initializer': ['normal', 'uniform']
    , 'activation': ['relu', 'elu', 'linear', 'softplus']
    , 'kernel_constraint': [0,1,2,3,4,5]
    , 'dropout_rate': [0, 0.25, 0.5, 0.75]
    , 'output_kernel_initializer': ['uniform']
    , 'output_activation': ['sigmoid']
    , 'loss': ['binary_crossentropy']
    , 'optimizer': ['Adadelta']
    , 'learning_rate': [1, 0.5, 0.7, 0.9]
    , 'momentum': [0.0, 0.25, 0.5]
    , 'rho': [0.95, 0.8, 0.9, 0.99]
    , 'beta_1': [99999]
    , 'beta_2': [99999]
    , 'return_metrics': ['False']
    , 'batch_size': [None, 32, 64, 128]
    , 'epochs': [1, 10, 100, 500, 1000]
}, 
# Adam optimizer
    {
    'num_neurons': [math.floor((len(keep_num_vars_gini)+1)/2)]
    , 'num_hidden_layers': [1, 2]
    , 'input_dim': [data['data_{}'.format(sample_values_solution[0])][keep_num_vars_gini].shape[1]]
    , 'kernel_initializer': ['normal', 'uniform']
    , 'activation': ['relu', 'elu', 'linear', 'softplus']
    , 'kernel_constraint': [0,1,2,3,4,5]
    , 'dropout_rate': [0, 0.25, 0.5, 0.75]
    , 'output_kernel_initializer': ['uniform']
    , 'output_activation': ['sigmoid']
    , 'loss': ['binary_crossentropy']
    , 'optimizer': ['Adam']
    , 'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3]
    , 'momentum': [99999]
    , 'rho': [99999]
    , 'beta_1': [0.9, 0.8, 0.95, 0.99]
    , 'beta_2': [0.999, 0.8, 0.9, 0.95]
    , 'return_metrics': ['False']
    , 'batch_size': [None, 32, 64, 128]
    , 'epochs': [1, 10, 100, 500, 1000]
}, 
# Nadam optimizer
    {
    'num_neurons': [math.floor((len(keep_num_vars_gini)+1)/2)]
    , 'num_hidden_layers': [1, 2]
    , 'input_dim': [data['data_{}'.format(sample_values_solution[0])][keep_num_vars_gini].shape[1]]
    , 'kernel_initializer': ['normal', 'uniform']
    , 'activation': ['relu', 'elu', 'linear', 'softplus']
    , 'kernel_constraint': [0,1,2,3,4,5]
    , 'dropout_rate': [0, 0.25, 0.5, 0.75]
    , 'output_kernel_initializer': ['uniform']
    , 'output_activation': ['sigmoid']
    , 'loss': ['binary_crossentropy']
    , 'optimizer': ['Nadam']
    , 'learning_rate': [0.002, 0.01, 0.1, 0.2]
    , 'momentum': [99999]
    , 'rho': [99999]
    , 'beta_1': [0.9, 0.8, 0.95, 0.99]
    , 'beta_2': [0.999, 0.8, 0.9, 0.95]
    , 'return_metrics': ['False']
    , 'batch_size': [None, 32, 64, 128]
    , 'epochs': [1, 10, 100, 500, 1000]
}, 
# Adamax optimizer
    {
    'num_neurons': [math.floor((len(keep_num_vars_gini)+1)/2)]
    , 'num_hidden_layers': [1, 2]
    , 'input_dim': [data['data_{}'.format(sample_values_solution[0])][keep_num_vars_gini].shape[1]]
    , 'kernel_initializer': ['normal', 'uniform']
    , 'activation': ['relu', 'elu', 'linear', 'softplus']
    , 'kernel_constraint': [0,1,2,3,4,5]
    , 'dropout_rate': [0, 0.25, 0.5, 0.75]
    , 'output_kernel_initializer': ['uniform']
    , 'output_activation': ['sigmoid']
    , 'loss': ['binary_crossentropy']
    , 'optimizer': ['Adamax']
    , 'learning_rate': [0.002, 0.01, 0.1, 0.2]
    , 'momentum': [99999]
    , 'rho': [99999]
    , 'beta_1': [0.9, 0.8, 0.95, 0.99]
    , 'beta_2': [0.999, 0.8, 0.9, 0.95]
    , 'return_metrics': ['False']
    , 'batch_size': [None, 32, 64, 128]
    , 'epochs': [1, 10, 100, 500, 1000]
}
]

params=[{'num_neurons': 4, 'num_hidden_layers': 2, 'input_dim': data['data_{}'.format(sample_values_solution[0])][keep_num_vars_gini].shape[1], 
             'kernel_initializer': 'normal', 'activation': 'relu', 'kernel_constraint': 2, 'dropout_rate': 0, 
             'output_kernel_initializer': 'uniform', 'output_activation': 'sigmoid', 'loss': 'binary_crossentropy', 
             'optimizer': 'SGD', 'learning_rate': 0.01, 'momentum': 0, 'rho': 99999, 'beta_1': 99999, 'beta_2': 99999, 
             'return_metrics': 'False', 'batch_size': None, 'epochs': 1}
        , {'num_neurons': 4, 'num_hidden_layers': 2, 'input_dim': data['data_{}'.format(sample_values_solution[0])][keep_num_vars_gini].shape[1], 
             'kernel_initializer': 'normal', 'activation': 'relu', 'kernel_constraint': 2, 'dropout_rate': 0, 
             'output_kernel_initializer': 'uniform', 'output_activation': 'sigmoid', 'loss': 'binary_crossentropy', 
             'optimizer': 'Adadelta', 'learning_rate': 1, 'momentum': 0.0, 'rho': 0.95, 'beta_1': 99999, 'beta_2': 99999, 
             'return_metrics': 'False', 'batch_size': None, 'epochs': 1}
        , {'num_neurons': 4, 'num_hidden_layers': 2, 'input_dim': data['data_{}'.format(sample_values_solution[0])][keep_num_vars_gini].shape[1], 
             'kernel_initializer': 'normal', 'activation': 'relu', 'kernel_constraint': 2, 'dropout_rate': 0, 
             'output_kernel_initializer': 'uniform', 'output_activation': 'sigmoid', 'loss': 'binary_crossentropy', 
             'optimizer': 'Adam', 'learning_rate': 0.001, 'momentum': 99999, 'rho': 99999, 'beta_1': 0.9, 'beta_2': 0.999, 
             'return_metrics': 'False', 'batch_size': None, 'epochs': 1}
        , {'num_neurons': 4, 'num_hidden_layers': 2, 'input_dim': data['data_{}'.format(sample_values_solution[0])][keep_num_vars_gini].shape[1], 
                     'kernel_initializer': 'normal', 'activation': 'relu', 'kernel_constraint': 2, 'dropout_rate': 0, 
                     'output_kernel_initializer': 'uniform', 'output_activation': 'sigmoid', 'loss': 'binary_crossentropy', 
                     'optimizer': 'Nadam', 'learning_rate': 0.002, 'momentum': 99999, 'rho': 99999, 'beta_1': 0.9, 'beta_2': 0.999, 
                     'return_metrics': 'False', 'batch_size': None, 'epochs': 1}
        , {'num_neurons': 4, 'num_hidden_layers': 2, 'input_dim': data['data_{}'.format(sample_values_solution[0])][keep_num_vars_gini].shape[1], 
                     'kernel_initializer': 'normal', 'activation': 'relu', 'kernel_constraint': 2, 'dropout_rate': 0, 
                     'output_kernel_initializer': 'uniform', 'output_activation': 'sigmoid', 'loss': 'binary_crossentropy', 
                     'optimizer': 'Adamax', 'learning_rate': 0.002, 'momentum': 99999, 'rho': 99999, 'beta_1': 0.9, 'beta_2': 0.999, 
                     'return_metrics': 'False', 'batch_size': None, 'epochs': 1}]

In [ ]:
opt_params_dict, loss_dict = mb.step_search_weight(estimator=KerasClassifier, 
                                                   params=params, 
                                                   grid=grid, 
                                                   target=target_variable_name,
                                                   weight=weight_variable_name, 
                                                   dev=data['data_{}'.format(sample_values_solution[0])], 
                                                   val=data['data_{}'.format(sample_values_solution[1])], 
                                                   keep=keep_num_vars_gini)

print('\n Best Parameters for each optimizer')
print(opt_params_dict)
print(loss_dict)

loss = loss_dict[loss_dict.index(min(loss_dict))]
opt_params = opt_params_dict[loss_dict.index(min(loss_dict))]
print('\n Best Parameters')
print(opt_params)
print(loss)

In [ ]:
# Select the top features
feats_best_keras = ks_fn.top_keras_feat(dev_df=data['data_{}'.format(sample_values_solution[0])], 
                                        feats=keep_num_vars_gini, 
                                        target=target_variable_name, 
                                        weight_variable=weight_variable_name, 
                                        threshold=0.0001, 
                                        keras_function=ks_fn.neural_network_function_wrapper(
    num_neurons_=[x[1] for x in list(opt_params.items()) if x[0]=='num_neurons'][0]
    , num_hidden_layers_=[x[1] for x in list(opt_params.items()) if x[0]=='num_hidden_layers'][0]
    , input_dim_=[x[1] for x in list(opt_params.items()) if x[0]=='input_dim'][0]
    , kernel_initializer_=[x[1] for x in list(opt_params.items()) if x[0]=='kernel_initializer'][0]
    , activation_=[x[1] for x in list(opt_params.items()) if x[0]=='activation'][0]
    , kernel_constraint_=[x[1] for x in list(opt_params.items()) if x[0]=='kernel_constraint'][0]
    , dropout_rate_=[x[1] for x in list(opt_params.items()) if x[0]=='dropout_rate'][0]
    , output_kernel_initializer_=[x[1] for x in list(opt_params.items()) if x[0]=='output_kernel_initializer'][0]
    , output_activation_=[x[1] for x in list(opt_params.items()) if x[0]=='output_activation'][0]
    , loss_=[x[1] for x in list(opt_params.items()) if x[0]=='loss'][0]
    , optimizer_=[x[1] for x in list(opt_params.items()) if x[0]=='optimizer'][0]
    , learning_rate_=[x[1] for x in list(opt_params.items()) if x[0]=='learning_rate'][0]
    , momentum_=[x[1] for x in list(opt_params.items()) if x[0]=='momentum'][0]
    , rho_=[x[1] for x in list(opt_params.items()) if x[0]=='rho'][0]
    , beta_1_=[x[1] for x in list(opt_params.items()) if x[0]=='beta_1'][0]
    , beta_2_=[x[1] for x in list(opt_params.items()) if x[0]=='beta_2'][0]
    , return_metrics_='True'), 
                epochs_=[x[1] for x in list(opt_params.items()) if x[0]=='epochs'][0]
              , batch_size_=[x[1] for x in list(opt_params.items()) if x[0]=='batch_size'][0]
            , feat_importance_num_display = 1000
                                 )

feats_best_keras

### Grid search based on iterative step search

In [ ]:
import math
from keras.wrappers.scikit_learn import KerasClassifier

grid = [
# SGD optimizer
    {
    'num_neurons': [math.floor((len(feats_best_keras)+1)/2)]
    , 'num_hidden_layers': [1, 2]
    , 'input_dim': [data['data_{}'.format(sample_values_solution[0])][feats_best_keras].shape[1]]
    , 'kernel_initializer': ['normal', 'uniform']
    , 'activation': ['relu', 'elu', 'linear', 'softplus']
    , 'kernel_constraint': [0,1,2,3,4,5]
    , 'dropout_rate': [0, 0.25, 0.5, 0.75]
    , 'output_kernel_initializer': ['uniform']
    , 'output_activation': ['sigmoid']
    , 'loss': ['binary_crossentropy']
    , 'optimizer': ['SGD']
    , 'learning_rate': [0.01, 0.001, 0.10, 0.30]
    , 'momentum': [0.0, 0.25, 0.5]
    , 'rho': [99999]
    , 'beta_1': [99999]
    , 'beta_2': [99999]
    , 'return_metrics': ['False']
    , 'batch_size': [None, 32, 64, 128]
    , 'epochs': [1, 10, 100, 500, 1000]
},
# Adadelta optimizer
    {
    'num_neurons': [math.floor((len(feats_best_keras)+1)/2)]
    , 'num_hidden_layers': [1, 2]
    , 'input_dim': [data['data_{}'.format(sample_values_solution[0])][feats_best_keras].shape[1]]
    , 'kernel_initializer': ['normal', 'uniform']
    , 'activation': ['relu', 'elu', 'linear', 'softplus']
    , 'kernel_constraint': [0,1,2,3,4,5]
    , 'dropout_rate': [0, 0.25, 0.5, 0.75]
    , 'output_kernel_initializer': ['uniform']
    , 'output_activation': ['sigmoid']
    , 'loss': ['binary_crossentropy']
    , 'optimizer': ['Adadelta']
    , 'learning_rate': [1, 0.5, 0.7, 0.9]
    , 'momentum': [0.0, 0.25, 0.5]
    , 'rho': [0.95, 0.8, 0.9, 0.99]
    , 'beta_1': [99999]
    , 'beta_2': [99999]
    , 'return_metrics': ['False']
    , 'batch_size': [None, 32, 64, 128]
    , 'epochs': [1, 10, 100, 500, 1000]
}, 
# Adam optimizer
    {
    'num_neurons': [math.floor((len(feats_best_keras)+1)/2)]
    , 'num_hidden_layers': [1, 2]
    , 'input_dim': [data['data_{}'.format(sample_values_solution[0])][feats_best_keras].shape[1]]
    , 'kernel_initializer': ['normal', 'uniform']
    , 'activation': ['relu', 'elu', 'linear', 'softplus']
    , 'kernel_constraint': [0,1,2,3,4,5]
    , 'dropout_rate': [0, 0.25, 0.5, 0.75]
    , 'output_kernel_initializer': ['uniform']
    , 'output_activation': ['sigmoid']
    , 'loss': ['binary_crossentropy']
    , 'optimizer': ['Adam']
    , 'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3]
    , 'momentum': [99999]
    , 'rho': [99999]
    , 'beta_1': [0.9, 0.8, 0.95, 0.99]
    , 'beta_2': [0.999, 0.8, 0.9, 0.95]
    , 'return_metrics': ['False']
    , 'batch_size': [None, 32, 64, 128]
    , 'epochs': [1, 10, 100, 500, 1000]
}, 
# Nadam optimizer
    {
    'num_neurons': [math.floor((len(feats_best_keras)+1)/2)]
    , 'num_hidden_layers': [1, 2]
    , 'input_dim': [data['data_{}'.format(sample_values_solution[0])][feats_best_keras].shape[1]]
    , 'kernel_initializer': ['normal', 'uniform']
    , 'activation': ['relu', 'elu', 'linear', 'softplus']
    , 'kernel_constraint': [0,1,2,3,4,5]
    , 'dropout_rate': [0, 0.25, 0.5, 0.75]
    , 'output_kernel_initializer': ['uniform']
    , 'output_activation': ['sigmoid']
    , 'loss': ['binary_crossentropy']
    , 'optimizer': ['Nadam']
    , 'learning_rate': [0.002, 0.01, 0.1, 0.2]
    , 'momentum': [99999]
    , 'rho': [99999]
    , 'beta_1': [0.9, 0.8, 0.95, 0.99]
    , 'beta_2': [0.999, 0.8, 0.9, 0.95]
    , 'return_metrics': ['False']
    , 'batch_size': [None, 32, 64, 128]
    , 'epochs': [1, 10, 100, 500, 1000]
}, 
# Adamax optimizer
    {
    'num_neurons': [math.floor((len(feats_best_keras)+1)/2)]
    , 'num_hidden_layers': [1, 2]
    , 'input_dim': [data['data_{}'.format(sample_values_solution[0])][feats_best_keras].shape[1]]
    , 'kernel_initializer': ['normal', 'uniform']
    , 'activation': ['relu', 'elu', 'linear', 'softplus']
    , 'kernel_constraint': [0,1,2,3,4,5]
    , 'dropout_rate': [0, 0.25, 0.5, 0.75]
    , 'output_kernel_initializer': ['uniform']
    , 'output_activation': ['sigmoid']
    , 'loss': ['binary_crossentropy']
    , 'optimizer': ['Adamax']
    , 'learning_rate': [0.002, 0.01, 0.1, 0.2]
    , 'momentum': [99999]
    , 'rho': [99999]
    , 'beta_1': [0.9, 0.8, 0.95, 0.99]
    , 'beta_2': [0.999, 0.8, 0.9, 0.95]
    , 'return_metrics': ['False']
    , 'batch_size': [None, 32, 64, 128]
    , 'epochs': [1, 10, 100, 500, 1000]
}
]

params=[{'num_neurons': 4, 'num_hidden_layers': 2, 'input_dim': data['data_{}'.format(sample_values_solution[0])][feats_best_keras].shape[1], 
             'kernel_initializer': 'normal', 'activation': 'relu', 'kernel_constraint': 2, 'dropout_rate': 0, 
             'output_kernel_initializer': 'uniform', 'output_activation': 'sigmoid', 'loss': 'binary_crossentropy', 
             'optimizer': 'SGD', 'learning_rate': 0.01, 'momentum': 0, 'rho': 99999, 'beta_1': 99999, 'beta_2': 99999, 
             'return_metrics': 'False', 'batch_size': None, 'epochs': 1}
        , {'num_neurons': 4, 'num_hidden_layers': 2, 'input_dim': data['data_{}'.format(sample_values_solution[0])][feats_best_keras].shape[1], 
             'kernel_initializer': 'normal', 'activation': 'relu', 'kernel_constraint': 2, 'dropout_rate': 0, 
             'output_kernel_initializer': 'uniform', 'output_activation': 'sigmoid', 'loss': 'binary_crossentropy', 
             'optimizer': 'Adadelta', 'learning_rate': 1, 'momentum': 0.0, 'rho': 0.95, 'beta_1': 99999, 'beta_2': 99999, 
             'return_metrics': 'False', 'batch_size': None, 'epochs': 1}
        , {'num_neurons': 4, 'num_hidden_layers': 2, 'input_dim': data['data_{}'.format(sample_values_solution[0])][feats_best_keras].shape[1], 
             'kernel_initializer': 'normal', 'activation': 'relu', 'kernel_constraint': 2, 'dropout_rate': 0, 
             'output_kernel_initializer': 'uniform', 'output_activation': 'sigmoid', 'loss': 'binary_crossentropy', 
             'optimizer': 'Adam', 'learning_rate': 0.001, 'momentum': 99999, 'rho': 99999, 'beta_1': 0.9, 'beta_2': 0.999, 
             'return_metrics': 'False', 'batch_size': None, 'epochs': 1}
        , {'num_neurons': 4, 'num_hidden_layers': 2, 'input_dim': data['data_{}'.format(sample_values_solution[0])][feats_best_keras].shape[1], 
                     'kernel_initializer': 'normal', 'activation': 'relu', 'kernel_constraint': 2, 'dropout_rate': 0, 
                     'output_kernel_initializer': 'uniform', 'output_activation': 'sigmoid', 'loss': 'binary_crossentropy', 
                     'optimizer': 'Nadam', 'learning_rate': 0.002, 'momentum': 99999, 'rho': 99999, 'beta_1': 0.9, 'beta_2': 0.999, 
                     'return_metrics': 'False', 'batch_size': None, 'epochs': 1}
        , {'num_neurons': 4, 'num_hidden_layers': 2, 'input_dim': data['data_{}'.format(sample_values_solution[0])][feats_best_keras].shape[1], 
                     'kernel_initializer': 'normal', 'activation': 'relu', 'kernel_constraint': 2, 'dropout_rate': 0, 
                     'output_kernel_initializer': 'uniform', 'output_activation': 'sigmoid', 'loss': 'binary_crossentropy', 
                     'optimizer': 'Adamax', 'learning_rate': 0.002, 'momentum': 99999, 'rho': 99999, 'beta_1': 0.9, 'beta_2': 0.999, 
                     'return_metrics': 'False', 'batch_size': None, 'epochs': 1}]

In [ ]:
opt_params_dict, loss_dict = mb.step_search_weight(estimator=KerasClassifier, 
                                                   params=params, 
                                                   grid=grid, 
                                                   target=target_variable_name,
                                                   weight=weight_variable_name, 
                                                   dev=data['data_{}'.format(sample_values_solution[0])], 
                                                   val=data['data_{}'.format(sample_values_solution[1])], 
                                                   keep=feats_best_keras)

print('\n Best Parameters for each optimizer')
print(opt_params_dict)
print(loss_dict)

loss = loss_dict[loss_dict.index(min(loss_dict))]
opt_params_iterative = opt_params_dict[loss_dict.index(min(loss_dict))]
print('\n Best Parameters')
print(opt_params_iterative)
print(loss)

In [ ]:
# Load NN machine leanring library
from keras.wrappers.scikit_learn import KerasClassifier

# Define the Keras model
nn = KerasClassifier(build_fn=ks_fn.neural_network_function_wrapper(
    num_neurons_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='num_neurons'][0]
    , num_hidden_layers_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='num_hidden_layers'][0]
    , input_dim_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='input_dim'][0]
    , kernel_initializer_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='kernel_initializer'][0]
    , activation_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='activation'][0]
    , kernel_constraint_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='kernel_constraint'][0]
    , dropout_rate_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='dropout_rate'][0]
    , output_kernel_initializer_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='output_kernel_initializer'][0]
    , output_activation_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='output_activation'][0]
    , loss_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='loss'][0]
    , optimizer_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='optimizer'][0]
    , learning_rate_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='learning_rate'][0]
    , momentum_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='momentum'][0]
    , rho_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='rho'][0]
    , beta_1_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='beta_1'][0]
    , beta_2_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='beta_2'][0]
    , return_metrics_='True'), verbose=0)

nn_model = nn.fit(data['data_{}'.format(sample_values_solution[0])][feats_best_keras].values, 
                  data['data_{}'.format(sample_values_solution[0])][target_variable_name], 
                  sample_weight=data['data_{}'.format(sample_values_solution[0])][weight_variable_name].values, 
                  use_multiprocessing=True, 
                  workers=8, 
                  verbose=0, 
                  epochs=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='epochs'][0], 
                  batch_size=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='batch_size'][0])

# Compute the score for the OOT data
for k in data.keys():
    data[k].loc[:, 'nn_score_iterative_numeric'] = nn.predict_proba(x = data[k][feats_best_keras])[:, 1]
    data[k].loc[:, 'nn_score_iterative_binary'] = list(map(round, data[k].loc[:, 'nn_score_iterative_numeric']))

### Grid search based on RandomizedSearchCV - also uses cross-validation

In [ ]:
import math

# define the grid search parameters
grid = [
# SGD optimizer
    {
    'batch_size': [None, 32, 64, 128]
    , 'epochs': [1, 10, 100, 500, 1000]
    , 'num_neurons': [math.floor((len(feats_best_keras)+1)/2)]
    , 'num_hidden_layers': [1, 2]
    , 'input_dim': [data['data_{}'.format(sample_values_solution[0])][feats_best_keras].shape[1]]
    , 'kernel_initializer': ['normal', 'uniform']
    , 'activation': ['relu', 'elu', 'linear', 'softplus']
    , 'kernel_constraint': [0,1,2,3,4,5]
    , 'dropout_rate': [0, 0.25, 0.5, 0.75]
    , 'output_kernel_initializer': ['uniform']
    , 'output_activation': ['sigmoid']
    , 'loss': ['binary_crossentropy']
    , 'optimizer': ['SGD']
    , 'learning_rate': [0.01, 0.001, 0.10, 0.30]
    , 'momentum': [0.0, 0.25, 0.5]
    , 'rho': [99999]
    , 'beta_1': [99999]
    , 'beta_2': [99999]
    , 'return_metrics': ['False']
},
# Adadelta optimizer
    {
    'batch_size': [None, 32, 64, 128]
    , 'epochs': [1, 10, 100, 500, 1000]
    , 'num_neurons': [math.floor((len(feats_best_keras)+1)/2)]
    , 'num_hidden_layers': [1, 2]
    , 'input_dim': [data['data_{}'.format(sample_values_solution[0])][feats_best_keras].shape[1]]
    , 'kernel_initializer': ['normal', 'uniform']
    , 'activation': ['relu', 'elu', 'linear', 'softplus']
    , 'kernel_constraint': [0,1,2,3,4,5]
    , 'dropout_rate': [0, 0.25, 0.5, 0.75]
    , 'output_kernel_initializer': ['uniform']
    , 'output_activation': ['sigmoid']
    , 'loss': ['binary_crossentropy']
    , 'optimizer': ['Adadelta']
    , 'learning_rate': [1, 0.5, 0.7, 0.9]
    , 'momentum': [0.0, 0.25, 0.5]
    , 'rho': [0.95, 0.8, 0.9, 0.99]
    , 'beta_1': [99999]
    , 'beta_2': [99999]
    , 'return_metrics': ['False']
}, 
# Adam optimizer
    {
    'batch_size': [None, 32, 64, 128]
    , 'epochs': [1, 10, 100, 500, 1000]
    , 'num_neurons': [math.floor((len(feats_best_keras)+1)/2)]
    , 'num_hidden_layers': [1, 2]
    , 'input_dim': [data['data_{}'.format(sample_values_solution[0])][feats_best_keras].shape[1]]
    , 'kernel_initializer': ['normal', 'uniform']
    , 'activation': ['relu', 'elu', 'linear', 'softplus']
    , 'kernel_constraint': [0,1,2,3,4,5]
    , 'dropout_rate': [0, 0.25, 0.5, 0.75]
    , 'output_kernel_initializer': ['uniform']
    , 'output_activation': ['sigmoid']
    , 'loss': ['binary_crossentropy']
    , 'optimizer': ['Adam']
    , 'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3]
    , 'momentum': [99999]
    , 'rho': [99999]
    , 'beta_1': [0.9, 0.8, 0.95, 0.99]
    , 'beta_2': [0.999, 0.8, 0.9, 0.95]
    , 'return_metrics': ['False']
}, 
# Nadam optimizer
    {
    'batch_size': [None, 32, 64, 128]
    , 'epochs': [1, 10, 100, 500, 1000]
    , 'num_neurons': [math.floor((len(feats_best_keras)+1)/2)]
    , 'num_hidden_layers': [1, 2]
    , 'input_dim': [data['data_{}'.format(sample_values_solution[0])][feats_best_keras].shape[1]]
    , 'kernel_initializer': ['normal', 'uniform']
    , 'activation': ['relu', 'elu', 'linear', 'softplus']
    , 'kernel_constraint': [0,1,2,3,4,5]
    , 'dropout_rate': [0, 0.25, 0.5, 0.75]
    , 'output_kernel_initializer': ['uniform']
    , 'output_activation': ['sigmoid']
    , 'loss': ['binary_crossentropy']
    , 'optimizer': ['Nadam']
    , 'learning_rate': [0.002, 0.01, 0.1, 0.2]
    , 'momentum': [99999]
    , 'rho': [99999]
    , 'beta_1': [0.9, 0.8, 0.95, 0.99]
    , 'beta_2': [0.999, 0.8, 0.9, 0.95]
    , 'return_metrics': ['False']
}, 
# Adamax optimizer
    {
    'batch_size': [None, 32, 64, 128]
    , 'epochs': [1, 10, 100, 500, 1000]
    , 'num_neurons': [math.floor((len(feats_best_keras)+1)/2)]
    , 'num_hidden_layers': [1, 2]
    , 'input_dim': [data['data_{}'.format(sample_values_solution[0])][feats_best_keras].shape[1]]
    , 'kernel_initializer': ['normal', 'uniform']
    , 'activation': ['relu', 'elu', 'linear', 'softplus']
    , 'kernel_constraint': [0,1,2,3,4,5]
    , 'dropout_rate': [0, 0.25, 0.5, 0.75]
    , 'output_kernel_initializer': ['uniform']
    , 'output_activation': ['sigmoid']
    , 'loss': ['binary_crossentropy']
    , 'optimizer': ['Adamax']
    , 'learning_rate': [0.002, 0.01, 0.1, 0.2]
    , 'momentum': [99999]
    , 'rho': [99999]
    , 'beta_1': [0.9, 0.8, 0.95, 0.99]
    , 'beta_2': [0.999, 0.8, 0.9, 0.95]
    , 'return_metrics': ['False']
}
]

kerasnn_grid_random_search = mb.grid_search_cv(
    n_splits = 2, # Number of cross-validation splits
    classifier = KerasClassifier, # Classifier name, e.g. RandomForestClassifier
    keras_function = ks_fn.neural_network_function_wrapper(
    num_neurons_=20
    , num_hidden_layers_=1
    , input_dim_=data['data_{}'.format(sample_values_solution[0])][keep_num_vars_gini].shape[1]
    , kernel_initializer_='normal'
    , activation_='relu'
    , kernel_constraint_=0
    , dropout_rate_=0.5
    , output_kernel_initializer_='normal'
    , output_activation_='sigmoid'
    , loss_='binary_crossentropy'
    , optimizer_='Adam'
    , learning_rate_=0.01
    , momentum_=0.0
    , rho_=0.95
    , beta_1_=0.9
    , beta_2_=0.999
    , return_metrics_='False'
),
    grid_params = grid, # Grid space
    dev_df = data['data_{}'.format(sample_values_solution[0])],  # Development sample that this will analysis will be performed
    feats = feats_best_keras, # List of predictor names
    target = target_variable_name, # Target variable name
    weight_variable = weight_variable_name, # Weight variable name
    randomized_search = True, # Set to True if randomized grid search will be performed, or to False if exhaustive grid search will be performed
    n_random_grids = 50, # Number of grid searches when randomized_search=True. If randomized_search=False, then this parameter is not applicable
    random_state = None, # If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator; If None, the random number generator is the RandomState instance used by np.random.
    n_jobs = 8 # Number of jobs to run in parallel
)

opt_params_fixed = kerasnn_grid_random_search.best_params_
print('The best hyperparameter set is:', opt_params_fixed)
print('Mean loss function for cross-validation test data: ', -kerasnn_grid_random_search.cv_results_['mean_test_score'][kerasnn_grid_random_search.best_index_])
print('Standard deviation loss function for cross-validation test data: ', kerasnn_grid_random_search.cv_results_['std_test_score'][kerasnn_grid_random_search.best_index_])

rp.plot_cross_validation_score(model=kerasnn_grid_random_search)

# Compute the score for the OOT data
for k in data.keys():
    data[k].loc[:, 'nn_score_random_numeric'] = kerasnn_grid_random_search.predict_proba(X = data[k][feats_best_keras])[:, 1]
    data[k].loc[:, 'nn_score_random_binary'] = list(map(round, data[k].loc[:, 'nn_score_random_numeric']))

### Grid search based on AutoKeras

In [ ]:
# DRAWBACKS
# 1) Autokeras does not optimize batch_size and epochs
# 2) Autokeras cannot incorporate weights

import timeit
start = timeit.default_timer()

import autokeras as ak

# Set the number of iterations
num_trials = 2

# Initialize the structured data classifier.
clf = ak.StructuredDataClassifier(column_names=feats_best_keras, loss='binary_crossentropy'
                                  , max_trials=num_trials
                                 , directory=r'C:\Data_Science123'                                  
) 

###############################################################################################################
# Data standardization with sklearn
from sklearn.preprocessing import StandardScaler
# create scaler
scaler = StandardScaler()
###############################################################################################################
# Data normalization with sklearn
#from sklearn.preprocessing import MinMaxScaler
# create scaler
#scaler = MinMaxScaler()
###############################################################################################################
# fit scaler on data
scaler.fit(data['data_{}'.format(sample_values_solution[0])][feats_best_keras].values.astype('float32'))
# apply transform
x_train = scaler.transform(data['data_{}'.format(sample_values_solution[0])][feats_best_keras].values.astype('float32'))
x_test = scaler.transform(data['data_{}'.format(sample_values_solution[1])][feats_best_keras].values.astype('float32'))

y_train = data['data_{}'.format(sample_values_solution[0])][target_variable_name]
y_test = data['data_{}'.format(sample_values_solution[1])][target_variable_name]

# Feed the structured data classifier with training data.
clf.fit(x=x_train, y=y_train
#        , sample_weight=data['data_{}'.format(sample_values_solution[0])][weight_variable_name].values
       , use_multiprocessing=True, workers=8, verbose=0
        , validation_data=(data['data_{}'.format(sample_values_solution[1])][feats_best_keras].values, 
                           data['data_{}'.format(sample_values_solution[1])][target_variable_name])
#              , epochs=[x[1] for x in list(opt_params_fixed.items()) if x[0]=='epochs'][0]
#              , batch_size=[x[1] for x in list(opt_params_fixed.items()) if x[0]=='batch_size'][0]       
       )

# Evaluate the best model with testing data.
autokeras_loss, autokeras_accuracy = clf.evaluate(x=data['data_{}'.format(sample_values_solution[1])][feats_best_keras].values, y=data['data_{}'.format(sample_values_solution[1])][target_variable_name], verbose=0)
print('Loss: %.3f' % autokeras_loss)
print('Accuracy: %.3f' % autokeras_accuracy)

# Evaluate the Auto-Keras model
from sklearn.metrics import classification_report
predictions = clf.predict(x=data['data_{}'.format(sample_values_solution[1])][feats_best_keras])
report = classification_report(data['data_{}'.format(sample_values_solution[1])][target_variable_name], predictions)
print(report)

stop = timeit.default_timer()
print('Execution time in seconds: ', stop - start)  
#Execution time in seconds:  202.19050749999587

## Calculate NN feature importance

In [ ]:
ks_fn.top_keras_feat(dev_df=data['data_{}'.format(sample_values_solution[0])], 
                     feats=feats_best_keras, 
                     target=target_variable_name, 
                     weight_variable=weight_variable_name, 
                     threshold=0, keras_function=ks_fn.neural_network_function_wrapper(
    num_neurons_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='num_neurons'][0]
    , num_hidden_layers_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='num_hidden_layers'][0]
    , input_dim_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='input_dim'][0]
    , kernel_initializer_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='kernel_initializer'][0]
    , activation_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='activation'][0]
    , kernel_constraint_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='kernel_constraint'][0]
    , dropout_rate_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='dropout_rate'][0]
    , output_kernel_initializer_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='output_kernel_initializer'][0]
    , output_activation_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='output_activation'][0]
    , loss_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='loss'][0]
    , optimizer_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='optimizer'][0]
    , learning_rate_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='learning_rate'][0]
    , momentum_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='momentum'][0]
    , rho_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='rho'][0]
    , beta_1_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='beta_1'][0]
    , beta_2_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='beta_2'][0]
    , return_metrics_='True'), 
                epochs_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='epochs'][0]
              , batch_size_=[x[1] for x in list(opt_params_iterative.items()) if x[0]=='batch_size'][0]
            , feat_importance_num_display = 1000
                                 )

## Produce NN reports

In [ ]:
nn_iterative_report_class = rp.binary_regression_report(
    predictions_dictionary = data, 
    target_variable = target_variable_name, 
    predicted_score_numeric = 'nn_score_iterative_numeric', 
    amount_variable_name = amount_variable_name_solution, 
    weight_variable_name = weight_variable_name_solution, 
    sample_values_dict = sample_values_dict, 
    select_top_percent = 100, 
    n_bands = 10, 
    rows = 10, 
    data_path = data_path
    )
nn_iterative_eval = nn_iterative_report_class.get_evaluation(predicted_score_binary = 'nn_score_iterative_binary', 
                                                       filename = 'evaluation_metrics_NN_iterative.csv')

In [ ]:
nn_random_report_class = rp.binary_regression_report(
    predictions_dictionary = data, 
    target_variable = target_variable_name, 
    predicted_score_numeric = 'nn_score_random_numeric', 
    amount_variable_name = amount_variable_name_solution, 
    weight_variable_name = weight_variable_name_solution, 
    sample_values_dict = sample_values_dict, 
    select_top_percent = 100, 
    n_bands = 10, 
    rows = 10, 
    data_path = data_path
    )
nn_random_eval = nn_random_report_class.get_evaluation(predicted_score_binary = 'nn_score_random_binary', 
                                                       filename = 'evaluation_metrics_NN_random.csv')

## Calculate NN Lifting table

In [ ]:
binary_report_class = nn_random_report_class

In [ ]:
# Create Lift table
binary_report_lift_table = binary_report_class.create_lift_table(filename = 'lift_table_NN_')

## Plots NN

### Plot NN Detection rate vs. Population Distribution

In [ ]:
# Create folder, if it doesn't exist
folder_name = 'graphs_NN'
ufun.create_folder(data_path = data_path, 
                   folder_name = 'output/{}'.format(folder_name))

In [ ]:
binary_report_class.plot_ADR_Quantile(
        folder_name = folder_name,
        xlim=None, 
        ylim=None
        )

### Plot NN Cum. Detection rate vs. Population Distribution (Gains chart)

In [ ]:
binary_report_class.plot_cADR_Quantile(
        folder_name = folder_name,
        xlim=None, 
        ylim=None
        )

### Plot NN FPR vs. Population Distribution

In [ ]:
binary_report_class.plot_FPR_Quantile(
        folder_name = folder_name,
        xlim=None, 
        ylim=None
        )

### Plot NN Cum. FPR vs. Population Distribution

In [ ]:
binary_report_class.plot_cFPR_Quantile(
        folder_name = folder_name,
        xlim=None, 
        ylim=None
        )

### Plot NN ROC curve

In [ ]:
binary_report_class.plot_ROC_curve(folder_name = folder_name)

### Plot NN Precision Recall curve

In [ ]:
binary_report_class.plot_precision_recall_curve(folder_name = folder_name)

### Plot NN F1 score, Accuracy, Sensitivity, Specificity, Precision vs Cutoff

In [ ]:
binary_report_class.plot_cutoffs(
        folder_name = folder_name,
        n_bands = 100, # Number of bands between 0 and 1
        return_table=False # Set to True in order to return the table that produced the graph, otherwise set to False
        )